#Reto Gobernacion del Santander

###Objetivo

Desarrollar tablero web con visualización geoespacial, análisis predictivo y chatbot comunitario

###Set up inicial

En esta sección montaremos Google Drive e importamos las librerias

In [ ]:
!pip install sodapy

In [ ]:
from google.colab import drive
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

from sodapy import Socrata
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, LabelEncoder

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

###Carga de datos

Cargaremos los 3 datasets de la Policía Nacional con la API de datos abiertos y visualizamos la informacion que hay dentro de ellos

In [ ]:
#Cargamos la base de datos (HURTOS) desde la API de datos abirtos.
client = Socrata(
    "www.datos.gov.co",
    None,  # Sin token
    timeout=180
)

print("\n")

# Descargar todos los datos y los guardamos en un DataFrame
results = client.get("d4fr-sbn2", limit=400000)
df_hurtos = pd.DataFrame.from_records(results)

# Le damos un vistazo a los datos de la BD
print(f" Total: {len(df_hurtos):,} registros")
print(f" Columnas: {list(df_hurtos.columns)}")
print("\n")
print(df_hurtos.head())

client.close()



 Total: 43,870 registros
 Columnas: ['departamento', 'municipio', 'codigo_dane', 'armas_medios', 'fecha_hecho', 'genero', 'grupo_etario', 'tipo_de_hurto', 'cantidad']


   departamento           municipio codigo_dane  armas_medios fecha_hecho  \
0       GUAJIRA  La Jagua del Pilar    44420000  NO REPORTADO  28/04/2024   
1     ANTIOQUIA               Andes    05034007  NO REPORTADO  18/02/2020   
2       GUAJIRA  San Juan del Cesar    44650000  NO REPORTADO  02/05/2020   
3  CUNDINAMARCA          Fusagasugá    25290000  NO REPORTADO  05/04/2019   
4     MAGDALENA           Aracataca    47053000  NO REPORTADO  07/08/2017   

      genero grupo_etario   tipo_de_hurto cantidad  
0  MASCULINO      ADULTOS  HURTO ABIGEATO        1  
1   FEMENINO      ADULTOS  HURTO ABIGEATO        1  
2  MASCULINO      ADULTOS  HURTO ABIGEATO        1  
3  MASCULINO      ADULTOS  HURTO ABIGEATO        1  
4   FEMENINO      ADULTOS  HURTO ABIGEATO        1  


In [ ]:
#Cargamos la base de datos (Delitos sexuales) desde la API de datos abirtos.
client = Socrata(
    "www.datos.gov.co",
    None,  # Sin token
    timeout=180
)

print("\n")

# Descargar todos los datos y los guardamos en un DataFrame
results = client.get("fpe5-yrmw", limit=400000)
df_delitos_sexuales = pd.DataFrame.from_records(results)

# Le damos un vistazo a los datos de la BD
print(f" Total: {len(df_delitos_sexuales):,} registros")
print(f" Columnas: {list(df_delitos_sexuales.columns)}")
print("\n")
print(df_delitos_sexuales.head())
print(df_delitos_sexuales["delito"].unique())
client.close()



 Total: 382,154 registros
 Columnas: ['departamento', 'municipio', 'codigo_dane', 'armas_medios', 'fecha_hecho', 'genero', 'grupo_etario', 'cantidad', 'delito']


  departamento   municipio codigo_dane  armas_medios fecha_hecho     genero  \
0     PUTUMAYO  Mocoa (CT)    86001000  NO REPORTADO  04/02/2010   FEMENINO   
1     CASANARE     Aguazul    85010000  NO REPORTADO  23/02/2010   FEMENINO   
2     CASANARE     Aguazul    85010000  NO REPORTADO  11/02/2010   FEMENINO   
3        VALLE     Palmira    76520000  NO REPORTADO  04/03/2010  MASCULINO   
4        VALLE     Palmira    76520000  NO REPORTADO  04/03/2010  MASCULINO   

   grupo_etario cantidad                                             delito  
0       ADULTOS        1                       ARTÍCULO 210 A. ACOSO SEXUAL  
1  ADOLESCENTES        1                 ARTÍCULO 206. ACTO SEXUAL VIOLENTO  
2       ADULTOS        1                 ARTÍCULO 206. ACTO SEXUAL VIOLENTO  
3  ADOLESCENTES        1  ARTÍCULO 211. ACTO SEX

In [ ]:
#Cargamos la base de datos (VIOLENCIA INTRAFAMILIAR) desde la API de datos abirtos.

client = Socrata(
    "www.datos.gov.co",
    None,  # Sin token
    timeout=180
)

print("\n")
# Descargar todos los datos y los guardamos en un DataFrame

results = client.get("vuyt-mqpw", limit=700000)
df_violencia = pd.DataFrame.from_records(results)


# Le damos un vistazo a los datos de la BD
print(f" Total: {len(df_violencia):,} registros")
print(f" Columnas: {list(df_violencia.columns)}")
print("\n")
print(df_violencia.head())

client.close()





 Total: 660,756 registros
 Columnas: ['departamento', 'municipio', 'codigo_dane', 'armas_medios', 'fecha_hecho', 'genero', 'grupo_etario', 'cantidad']


         departamento         municipio codigo_dane         armas_medios  \
0           ANTIOQUIA     Medellín (CT)    05001000  SIN EMPLEO DE ARMAS   
1        CUNDINAMARCA  Bogotá D.C. (CT)    11001000  SIN EMPLEO DE ARMAS   
2        CUNDINAMARCA  Bogotá D.C. (CT)    11001000  SIN EMPLEO DE ARMAS   
3               CAUCA           Cajibío    19130000  SIN EMPLEO DE ARMAS   
4  NORTE DE SANTANDER           Teorama    54800000  SIN EMPLEO DE ARMAS   

  fecha_hecho     genero grupo_etario cantidad  
0  18/09/2025  MASCULINO      MENORES        1  
1  23/10/2025   FEMENINO      MENORES        5  
2  23/10/2025  MASCULINO      MENORES        5  
3  23/10/2025   FEMENINO      ADULTOS        1  
4  06/10/2025  MASCULINO      ADULTOS        1  


In [ ]:
#Cargamos la base de datos (VIOLENCIA INTRAFAMILIAR) desde la API de datos abirtos.

client = Socrata(
    "www.datos.gov.co",
    None,  # Sin token
    timeout=180
)

print("\n")

# Descargar todos los datos y los guardamos en un DataFrame
results = client.get("x46e-abhz", limit=200000)
df_bucaramanga = pd.DataFrame.from_records(results)


# Le damos un vistazo a los datos de la BD
print(f" Total: {len(df_bucaramanga):,} registros")
print(f" Columnas: {list(df_bucaramanga.columns)}")
print(df_bucaramanga.head())

client.close()




 Total: 120,940 registros
 Columnas: ['descripcion_conducta', 'armas_medios', 'barrios_hecho', 'fecha_hecho', 'hora_hecho', 'edad', 'sexo', 'movil_victima', 'movil_agresor', 'clase_sitio', 'articulo', 'delito_solo', 'curso_vida', 'curso_vida_orden', 'a_o_num', 'mes_num', 'dia_num', 'rango_horario', 'tipolog_a', 'rango_horario_orden', 'dia_nombre', 'dia_nombre_orden', 'localidad', 'num_com', 'nom_com', 'cantidad_unica']
                descripcion_conducta                 armas_medios  \
0  ARTÍCULO 111. LESIONES PERSONALES  ARMA BLANCA / CORTOPUNZANTE   
1  ARTÍCULO 111. LESIONES PERSONALES  ARMA BLANCA / CORTOPUNZANTE   
2  ARTÍCULO 111. LESIONES PERSONALES  ARMA BLANCA / CORTOPUNZANTE   
3  ARTÍCULO 111. LESIONES PERSONALES  ARMA BLANCA / CORTOPUNZANTE   
4  ARTÍCULO 111. LESIONES PERSONALES  ARMA BLANCA / CORTOPUNZANTE   

  barrios_hecho              fecha_hecho hora_hecho edad       sexo  \
0    TRANSICION  2016-01-01T00:00:00.000   20:00:00   54  MASCULINO   
1    ESTORAQUES  2

In [ ]:
# Crear lista de tuplas (provincia, municipio)
provincias_municipios = [
    # Provincia Comunera
    ('Comunera', 'Chima'),
    ('Comunera', 'Confines'),
    ('Comunera', 'Contratación'),
    ('Comunera', 'El Guacamayo'),
    ('Comunera', 'Galán'),
    ('Comunera', 'Gámbita'),
    ('Comunera', 'Guadalupe'),
    ('Comunera', 'Guapotá'),
    ('Comunera', 'Hato'),
    ('Comunera', 'Oiba'),
    ('Comunera', 'Palmar'),
    ('Comunera', 'Palmas del Socorro'),
    ('Comunera', 'Santa Helena del Opón'),
    ('Comunera', 'Simacota'),
    ('Comunera', 'Socorro'),
    ('Comunera', 'Suaita'),

    # Provincia García-Rovira
    ('García-Rovira', 'Capitanejo'),
    ('García-Rovira', 'Carcasí'),
    ('García-Rovira', 'Cepitá'),
    ('García-Rovira', 'Cerrito'),
    ('García-Rovira', 'Concepción'),
    ('García-Rovira', 'Enciso'),
    ('García-Rovira', 'Guaca'),
    ('García-Rovira', 'Macaravita'),
    ('García-Rovira', 'Málaga'),
    ('García-Rovira', 'Molagavita'),
    ('García-Rovira', 'San Andrés'),
    ('García-Rovira', 'San José de Miranda'),
    ('García-Rovira', 'San Miguel'),

    # Provincia Guanentá
    ('Guanentá', 'Aratoca'),
    ('Guanentá', 'Barichara'),
    ('Guanentá', 'Cabrera'),
    ('Guanentá', 'Coromoro'),
    ('Guanentá', 'Curití'),
    ('Guanentá', 'Charalá'),
    ('Guanentá', 'Encino'),
    ('Guanentá', 'Jordán'),
    ('Guanentá', 'Mogotes'),
    ('Guanentá', 'Ocamonte'),
    ('Guanentá', 'Onzaga'),
    ('Guanentá', 'Páramo'),
    ('Guanentá', 'Pinchote'),
    ('Guanentá', 'San Joaquín'),
    ('Guanentá', 'San Gil'),
    ('Guanentá', 'Valle de San José'),
    ('Guanentá', 'Villanueva'),

    # Provincia Soto Norte
    ('Soto Norte', 'California'),
    ('Soto Norte', 'Charta'),
    ('Soto Norte', 'Matanza'),
    ('Soto Norte', 'Suratá'),
    ('Soto Norte', 'Tona'),
    ('Soto Norte', 'Vetas'),

    # Provincia Vélez
    ('Vélez', 'Aguada'),
    ('Vélez', 'Albania'),
    ('Vélez', 'Barbosa'),
    ('Vélez', 'Bolívar'),
    ('Vélez', 'Cimitarra'),
    ('Vélez', 'El Peñón'),
    ('Vélez', 'Chipatá'),
    ('Vélez', 'Florián'),
    ('Vélez', 'Guavatá'),
    ('Vélez', 'Güepsa'),
    ('Vélez', 'Jesús María'),
    ('Vélez', 'La Belleza'),
    ('Vélez', 'La Paz'),
    ('Vélez', 'Landázuri'),
    ('Vélez', 'Puente Nacional'),
    ('Vélez', 'Puerto Parra'),
    ('Vélez', 'San Benito'),
    ('Vélez', 'Sucre'),
    ('Vélez', 'Vélez'),

    # Provincia Yariguíes
    ('Yariguíes', 'Barrancabermeja'),
    ('Yariguíes', 'Betulia'),
    ('Yariguíes', 'El Carmen de Chucurí'),
    ('Yariguíes', 'Puerto Wilches'),
    ('Yariguíes', 'Sabana de Torres'),
    ('Yariguíes', 'San Vicente de Chucurí'),

    # Provincia Metropolitana
    ('Metropolitana', 'Bucaramanga'),
    ('Metropolitana', 'El Playón'),
    ('Metropolitana', 'Floridablanca'),
    ('Metropolitana', 'Girón'),
    ('Metropolitana', 'Lebrija'),
    ('Metropolitana', 'Los Santos'),
    ('Metropolitana', 'Piedecuesta'),
    ('Metropolitana', 'Rionegro'),
    ('Metropolitana', 'Santa Bárbara'),
    ('Metropolitana', 'Zapatoca'),
]

# Crear DataFrame
df_provincias = pd.DataFrame(provincias_municipios, columns=['ZONA', 'MUNICIPIO'])

# Mostrar primeras filas
print(df_provincias.head(10))
print(f"\nTotal de municipios: {len(df_provincias)}")
print(f"Total de provincias: {df_provincias['ZONA'].nunique()}")

       ZONA     MUNICIPIO
0  Comunera         Chima
1  Comunera      Confines
2  Comunera  Contratación
3  Comunera  El Guacamayo
4  Comunera         Galán
5  Comunera       Gámbita
6  Comunera     Guadalupe
7  Comunera       Guapotá
8  Comunera          Hato
9  Comunera          Oiba

Total de municipios: 87
Total de provincias: 7


##Preprocesamiento de datos


### Estandarizamos los 3 datasets de la policia nacional ya que se van a trabajar juntos

####Estandarizacion de los nombres de las columnas en los datasets

In [ ]:
#Introducimos un nuevo diccionario para estandarizar los nombres de las columnas
columnas_dict = {
    # Ubicación geográfica
    'departamento': 'DEPARTAMENTO',
    'dpto': 'DEPARTAMENTO',
    'municipio': 'MUNICIPIO',
    'nom_mpio': 'MUNICIPIO',
    'codigo_dane': 'CODIGO DANE',
    'cod_dpto': 'CODIGO DEPARTAMENTO',
    'cod_mpio': 'CODIGO MUNICIPIO',
    'tipo_municipio': 'TIPO MUNICIPIO',
    'localidad': 'LOCALIDAD',
    'barrios_hecho': 'BARRIO',
    'num_com': 'NUMERO COMUNA',
    'nom_com': 'NOMBRE COMUNA',

    # Coordenadas
    'longitud': 'LONGITUD',
    'latitud': 'LATITUD',

    # Información temporal
    'fecha_hecho': 'FECHA HECHO',
    'hora_hecho': 'HORA HECHO',
    'a_o_num': 'AÑO',
    'mes_num': 'MES',
    'dia_num': 'DIA',
    'dia_nombre': 'DIA NOMBRE',
    'dia_nombre_orden': 'DIA NOMBRE ORDEN',
    'rango_horario': 'RANGO HORARIO',
    'rango_horario_orden': 'RANGO HORARIO ORDEN',

    # Información del delito
    'delito': 'DELITO',
    'delito_solo': 'DELITO',
    'descripcion_conducta': 'DESCRIPCION CONDUCTA',
    'tipo_de_hurto': 'TIPO HURTO',
    'tipolog_a': 'TIPOLOGIA',
    'articulo': 'ARTICULO',
    'clase_sitio': 'CLASE SITIO',

    # Armas y medios
    'armas_medios': 'ARMAS MEDIOS',

    # Información de la víctima
    'genero': 'GENERO',
    'sexo': 'GENERO',
    'grupo_etario': 'GRUPO ETARIO',
    'edad': 'EDAD',
    'curso_vida': 'CURSO VIDA',
    'curso_vida_orden': 'CURSO VIDA ORDEN',

    # Movilidad
    'movil_victima': 'MOVIL VICTIMA',
    'movil_agresor': 'MOVIL AGRESOR',

    # Conteos
    'cantidad': 'CANTIDAD',
    'cantidad_unica': 'CANTIDAD'
}

In [ ]:
#Cambiamos los nombres de las columnas para entenderlos mejor y estandarizarlos
df_hurtos = df_hurtos.rename(columns=columnas_dict)
df_delitos_sexuales = df_delitos_sexuales.rename(columns=columnas_dict)
df_violencia = df_violencia.rename(columns=columnas_dict)
df_bucaramanga = df_bucaramanga.rename(columns=columnas_dict)


####Estandarizamos los datos de la columna armas para evitar duplicidad de datos con respecto a sinonimos que pueden aparecer

In [ ]:
#Estandarizamos la columna armas creando un diccionario
estandarizacion_armas = {
    'NO REPORTADO': 'NO REPORTADO',
    'ARMA DE FUEGO': 'ARMA DE FUEGO',
    'ESCOPOLAMINA': 'ESCOPOLAMINA',
    'SIN EMPLEO DE ARMAS': 'SIN EMPLEO DE ARMAS',
    'CONTUNDENTES': 'CONTUNDENTES',
    'LLAVE MAESTRA': 'LLAVE MAESTRA',
    'PALANCAS': 'PALANCAS',
    'LICOR ADULTERADO': 'LICOR ADULTERADO',
    'CINTAS/CINTURON': 'CINTAS/CINTURON',
    'ESPOSAS': 'ESPOSAS',
    'ARTEFACTO EXPLOSIVO/CARGA DINAMITA': 'ARTEFACTO EXPLOSIVO',
    'ARMA BLANCA / CORTOPUNZANTE': 'ARMA BLANCA',
    'ARMAS BLANCAS': 'ARMA BLANCA',
    'CORTOPUNZANTES': 'ARMA BLANCA',
    'CORTANTES': 'ARMA BLANCA',
    'PUNZANTES': 'ARMA BLANCA',
}


In [ ]:
#Cambiamos los datos de la columna para evitar duplicidad y estandarizarlos
df_delitos_sexuales['ARMAS MEDIOS'] = df_delitos_sexuales['ARMAS MEDIOS'].replace(estandarizacion_armas)
df_hurtos['ARMAS MEDIOS'] = df_hurtos['ARMAS MEDIOS'].replace(estandarizacion_armas)
df_violencia['ARMAS MEDIOS'] = df_violencia['ARMAS MEDIOS'].replace(estandarizacion_armas)


####Estandarizamos los datos en la columna genero para evitar posibles sesgos.

In [ ]:
# Estandarizacion de genero
estandarizacion_genero = {
    'MASCULINO': 'MASCULINO',
    'FEMENINO': 'FEMENINO',
    'NO REPORTADO': 'NO REPORTADO',
    'NO REPORTA': 'NO REPORTADO'  # Estandarizar a NO REPORTADO
}


In [ ]:
df_delitos_sexuales['GENERO'] = df_delitos_sexuales['GENERO'].replace(estandarizacion_genero)
df_hurtos['GENERO'] = df_hurtos['GENERO'].replace(estandarizacion_genero)
df_violencia['GENERO'] = df_violencia['GENERO'].replace(estandarizacion_genero)

####Estandarizamos los datos en detilo y tipo de hurto para crear una columna llamada tipo de delito

In [ ]:
estandarizacion_delitos_sexuales = {
    # Acceso carnal con menores
    'ARTÍCULO 208. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS': 'ACCESO CARNAL CON MENOR',
    'ARTÍCULO 211. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS (CIRCUNSTANCIAS AGRAVACIÓN)': 'ACCESO CARNAL CON MENOR',

    # Actos sexuales con menores
    'ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS': 'ACTOS SEXUALES CON MENOR',
    'ARTÍCULO 211. ACTOS SEXUALES CON MENOR DE 14 AÑOS (CIRCUNSTANCIAS DE AGRAVACIÓN)': 'ACTOS SEXUALES CON MENOR',

    # Acceso carnal violento
    'ARTÍCULO 205. ACCESO CARNAL VIOLENTO': 'ACCESO CARNAL VIOLENTO',
    'ARTÍCULO 211. ACCESO CARNAL VIOLENTO (CIRCUNSTANCIAS AGRAVACIÓN)': 'ACCESO CARNAL VIOLENTO',

    # Acto sexual violento
    'ARTÍCULO 206. ACTO SEXUAL VIOLENTO': 'ACTO SEXUAL VIOLENTO',
    'ARTÍCULO 211. ACTO SEXUAL VIOLENTO (CIRCUNSTANCIAS DE AGRAVACIÓN)': 'ACTO SEXUAL VIOLENTO',

    # Acceso/acto con incapaz
    'ARTÍCULO 210. ACCESO CARNAL O ACTO SEXUAL ABUSIVO CON INCAPAZ DE RESISTIR': 'ABUSO A PERSONA INCAPAZ',
    'ARTÍCULO 207. ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR': 'ABUSO A PERSONA INCAPAZ',
    'ARTÍCULO 211. ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR  (CIRCUNSTANC': 'ABUSO A PERSONA INCAPAZ',
    'ARTÍCULO 211. ACCESO CARNAL O ACTO SEXUAL ABUSIVO CON INCAPAZ DE RESISTIR (CIRCUNSTANCIAS AGRAVACIÓN': 'ABUSO A PERSONA INCAPAZ',

    # Acoso sexual
    'ARTÍCULO 210 A. ACOSO SEXUAL': 'ACOSO SEXUAL',

    # Pornografía y explotación de menores
    'ARTÍCULO 218. PORNOGRAFÍA CON MENORES': 'EXPLOTACION SEXUAL MENORES',
    'ARTÍCULO 219 A. UTILIZACIÓN O FACILITACIÓN DE MEDIOS DE COMUNICACIÓN PARA OFRECER SERVICIOS SEXUALES DE MENORES': 'EXPLOTACION SEXUAL MENORES',
    'ARTÍCULO 217 A. DEMANDA DE EXPLOTACION SEXUAL COMERCIAL DE PERSONA MENOR DE 18 AÑOS DE EDAD': 'EXPLOTACION SEXUAL MENORES',
    'ARTÍCULO 217. ESTÍMULO A LA PROSTITUCIÓN DE MENORES': 'EXPLOTACION SEXUAL MENORES',
    'ARTÍCULO 213 A. PROXENETISMO CON MENOR DE EDAD': 'EXPLOTACION SEXUAL MENORES',

    # Prostitución forzada
    'ARTÍCULO 213. INDUCCIÓN A LA PROSTITUCIÓN': 'PROSTITUCION FORZADA',
    'ARTÍCULO 214. CONSTREÑIMIENTO A LA PROSTITUCIÓN': 'PROSTITUCION FORZADA',
    'ARTÍCULO 216. INDUCCIÓN A LA PROSTITUCIÓN (CIRCUNSTANCIAS AGRAVACIÓN)': 'PROSTITUCION FORZADA',
    'ARTÍCULO 216. CONSTREÑIMIENTO A LA PROSTITUCIÓN (CIRCUNSTANCIAS AGRAVACIÓN)': 'PROSTITUCION FORZADA',

    # Omisión de denuncia
    'ARTÍCULO 219 B. OMISIÓN DE DENUNCIA': 'OMISION DE DENUNCIA',
}

In [ ]:
estandarizacion_hurtos = {
    'HURTO ABIGEATO': 'ABIGEATO',
    'HURTO PIRATERÍA TERRESTRE': 'PIRATERIA TERRESTRE',
    'HURTO ENTIDADES FINANCIERAS': 'ENTIDADES FINANCIERAS'
}


####Insertamos 3 nuevas columnas en los datasets para tener las mismas columnas al momento de unirlos

In [ ]:
#Creamos columnas nuevas para almacenar las caracteristicas nuevas extraidas de la base de datos.
df_delitos_sexuales['CATEGORIA DELITO'] = 'DELITO SEXUAL'
df_delitos_sexuales['TIPO DELITO'] = df_delitos_sexuales['DELITO'].replace(estandarizacion_delitos_sexuales)
df_delitos_sexuales['DELITO DETALLADO'] = df_delitos_sexuales['DELITO']

In [ ]:
#Creamos columnas nuevas para almacenar las caracteristicas nuevas extraidas de la base de datos.
df_hurtos['CATEGORIA DELITO'] = 'HURTO'
df_hurtos['TIPO DELITO'] = df_hurtos['TIPO HURTO'].replace(estandarizacion_hurtos)
df_hurtos['DELITO DETALLADO'] = df_hurtos['TIPO HURTO']

In [ ]:
#Creamos columnas nuevas para almacenar las caracteristicas nuevas extraidas de la base de datos.
df_violencia['CATEGORIA DELITO'] = 'VIOLENCIA INTRAFAMILIAR'
df_violencia['TIPO DELITO'] = 'VIOLENCIA INTRAFAMILIAR'
df_violencia['DELITO DETALLADO'] = 'VIOLENCIA INTRAFAMILIAR'

####Datasets estandarizados

In [ ]:
print(df_hurtos.info())
print(df_delitos_sexuales.info())
print(df_violencia.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43870 entries, 0 to 43869
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DEPARTAMENTO      43870 non-null  object
 1   MUNICIPIO         43870 non-null  object
 2   CODIGO DANE       43870 non-null  object
 3   ARMAS MEDIOS      43870 non-null  object
 4   FECHA HECHO       43870 non-null  object
 5   GENERO            39650 non-null  object
 6   GRUPO ETARIO      42859 non-null  object
 7   TIPO HURTO        43870 non-null  object
 8   CANTIDAD          43870 non-null  object
 9   CATEGORIA DELITO  43870 non-null  object
 10  TIPO DELITO       43870 non-null  object
 11  DELITO DETALLADO  43870 non-null  object
dtypes: object(12)
memory usage: 4.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382154 entries, 0 to 382153
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ---

In [ ]:
# Estandarizamos municipios
df_hurtos['MUNICIPIO'] = df_hurtos['MUNICIPIO'].str.upper().replace('(CT)', '', regex=False)
df_hurtos['MUNICIPIO'] = df_hurtos['MUNICIPIO'].str.replace('(CT)', '', regex=False)
df_delitos_sexuales['MUNICIPIO'] = df_delitos_sexuales['MUNICIPIO'].str.upper()
df_delitos_sexuales['MUNICIPIO'] = df_delitos_sexuales['MUNICIPIO'].str.replace('(CT)', '', regex=False)
df_violencia['MUNICIPIO'] = df_violencia['MUNICIPIO'].str.replace('(CT)', '', regex=False)
df_violencia['MUNICIPIO'] = df_violencia['MUNICIPIO'].str.upper()
df_provincias['MUNICIPIO'] = df_provincias['MUNICIPIO'].str.upper()
df_provincias["ZONA"]=df_provincias["ZONA"].str.upper()
print(df_delitos_sexuales['MUNICIPIO'].unique())
print(df_violencia['MUNICIPIO'].unique())
print(df_hurtos['MUNICIPIO'].unique())

print(df_hurtos.head())


['MOCOA ' 'AGUAZUL' 'PALMIRA' ... nan 'RECETOR' 'JORDÁN']
['MEDELLÍN ' 'BOGOTÁ D.C. ' 'CAJIBÍO' ... 'SAN BENITO' 'ZAPAYÁN' 'CEPITÁ']
['LA JAGUA DEL PILAR' 'ANDES' 'SAN JUAN DEL CESAR' 'FUSAGASUGÁ'
 'ARACATACA' 'AGUSTÍN CODAZZI' 'BARRANCA DE UPÍA' 'MANATÍ'
 'SAN JOSÉ DEL GUAVIARE ' 'MONTERREY' 'SAN ALBERTO' 'FUNDACIÓN'
 'SAN MARCOS' 'SANTA BÁRBARA DE PINTO' 'PLANETA RICA' 'MISTRATÓ' 'YOPAL '
 'CHINÚ' 'LORICA' 'TÁMARA' 'PAICOL' 'SAN JOSÉ DE URÉ' 'MONTELÍBANO'
 'REMEDIOS' 'BUENAVISTA' 'SANTA ROSA DE OSOS' 'CAUCASIA' 'RIONEGRO'
 'CHIGORODÓ' 'CAROLINA' 'URRAO' 'ALBÁN' 'EL CARMEN DE VIBORAL' 'LA CEJA'
 'SARDINATA' 'SAN ANTERO' 'BETULIA' 'CIUDAD BOLÍVAR' 'SAHAGÚN'
 'PUEBLO NUEVO' 'SANTA ROSALÍA' 'ANSERMA' 'MOMIL' 'SONSON' 'FLORENCIA '
 'VILLAVICENCIO ' 'IBAGUÉ ' 'PIVIJAY' 'MONGUA' 'BOGOTÁ D.C. ' 'SAN PEDRO'
 'CURILLO' 'SAN VICENTE' 'VICTORIA' 'PORE' 'LA CALERA'
 'VILLA DE SAN DIEGO DE UBATE' 'EL SANTUARIO' 'YOTOCO' 'COTA' 'FACATATIVÁ'
 'ANSERMANUEVO' 'DAGUA' 'CARTAGO' 'GUADALAJARA DE BUGA' 'M

In [ ]:
#eliminar la columna delito
df_hurtos.drop(columns=['TIPO HURTO'], inplace=True)
df_delitos_sexuales.drop(columns=['DELITO'], inplace=True)

###Estandarizacion del dataset de bucaramanga

In [ ]:
# Vamos a mostrar todos los unicos en las columnas objeto
for column in df_bucaramanga.select_dtypes(include='object').columns:
    print(f"Hay {len(df_bucaramanga[column].unique())} items unicos en {column}")
    print(df_bucaramanga[column].unique())
    print("\n")

Hay 44 items unicos en DESCRIPCION CONDUCTA
['ARTÍCULO 111. LESIONES PERSONALES'
 'ARTÍCULO 229. VIOLENCIA INTRAFAMILIAR' 'ARTÍCULO 239. HURTO AUTOMOTORES'
 'ARTÍCULO 239. HURTO ENTIDADES COMERCIALES'
 'ARTÍCULO 239. HURTO MOTOCICLETAS' 'ARTÍCULO 239. HURTO PERSONAS'
 'ARTÍCULO 239. HURTO RESIDENCIAS' 'ARTÍCULO 244. EXTORSIÓN'
 'ARTÍCULO 208. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS'
 'ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS'
 'ARTÍCULO 103. HOMICIDIO' 'ARTÍCULO 205. ACCESO CARNAL VIOLENTO'
 'ARTÍCULO 206. ACTO SEXUAL VIOLENTO'
 'ARTÍCULO 210. ACCESO CARNAL O ACTO SEXUAL ABUSIVO CON INCAPAZ DE RESISTIR'
 'ARTÍCULO 213. INDUCCIÓN A LA PROSTITUCIÓN'
 'ARTÍCULO 219 A. UTILIZACIÓN O FACILITACIÓN DE MEDIOS DE COMUNICACIÓN PARA OFRECER SERVICIOS SEXUALES DE MENORES'
 'ARTÍCULO 210 A. ACOSO SEXUAL' 'ARTÍCULO 104A. FEMINICIDIO'
 'ARTÍCULO 207. ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR'
 'ARTÍCULO 211. ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN I

####Estandarizamos los delitos de bucaramanga

####Estandarizamos las armas del dataset de bucaramanga

In [ ]:
# ESTANDARIZACIÓN DE ARMAS Y MEDIOS

estandarizacion_armas_bga = {
    'NO DISPONIBLE': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO',
    'ARMA DE FUEGO': 'ARMA DE FUEGO',
    'ESCOPOLAMINA': 'ESCOPOLAMINA',
    'SIN EMPLEO DE ARMAS': 'SIN EMPLEO DE ARMAS',
    'CONTUNDENTES': 'CONTUNDENTES',
    'LLAVE MAESTRA': 'LLAVE MAESTRA',
    'PALANCAS': 'PALANCAS',
    'LICOR ADULTERADO': 'LICOR ADULTERADO',
    'ARTEFACTO EXPLOSIVO/CARGA DINAMITA': 'ARTEFACTO EXPLOSIVO',
    'PAPA EXPLOSIVA': 'ARTEFACTO EXPLOSIVO',
    'GRANADA DE MANO': 'ARTEFACTO EXPLOSIVO',
    'ARMA BLANCA / CORTOPUNZANTE': 'ARMA BLANCA',
    'CORTOPUNZANTES': 'ARMA BLANCA',
    'CORTANTES': 'ARMA BLANCA',
    'PUNZANTES': 'ARMA BLANCA',
    'CUERDA/SOGA/CADENA': 'CUERDA/SOGA/CADENA',
    'LLAMADA TELEFONICA': 'LLAMADA TELEFONICA',
    'CARTA EXTORSIVA': 'CARTA EXTORSIVA',
    'REDES SOCIALES': 'REDES SOCIALES',
    'DIRECTA': 'DIRECTA',
    'MIXTA': 'MIXTA',
    'PERRO': 'ANIMAL',
    'QUIMICOS': 'SUSTANCIAS QUIMICAS',
    'VENENO': 'SUSTANCIAS QUIMICAS',
    'ACIDO': 'SUSTANCIAS QUIMICAS',
    'SUSTANCIAS TOXICAS': 'SUSTANCIAS QUIMICAS',
    'GASES': 'SUSTANCIAS QUIMICAS',
    'MEDICAMENTOS': 'MEDICAMENTOS',
    'ALUCINOGENOS': 'MEDICAMENTOS',
    'ALIMENTOS VENCIDOS': 'ALIMENTOS VENCIDOS',
    'BOLSA PLASTICA': 'ASFIXIA',
    'JERINGA': 'JERINGA',
    'VEHICULO': 'VEHICULO',
    'MOTO': 'MOTOCICLETA',
    'BICICLETA': 'BICICLETA',
    'ARTEFACTO INCENDIARIO': 'INCENDIARIO',
    'COMBUSTIBLE': 'INCENDIARIO',
    'AGUA CALIENTE': 'AGUA CALIENTE',
    'PRENDAS DE VESTIR': 'PRENDAS DE VESTIR',
    'ARMA TRAUMATICA': 'ARMA TRAUMATICA',
}

In [ ]:
df_bucaramanga['ARMAS MEDIOS'] = df_bucaramanga['ARMAS MEDIOS'].replace(estandarizacion_armas_bga)

####Estandarizacion de genero de bucaramanga

In [ ]:
# ESTANDARIZACIÓN DE GÉNERO

estandarizacion_genero_bga = {
    'MASCULINO': 'MASCULINO',
    'FEMENINO': 'FEMENINO',
    'NO DISPONIBLE': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO',
    'NO REPORTA': 'NO REPORTADO',}

In [ ]:
df_bucaramanga['GENERO'] = df_bucaramanga['GENERO'].replace(estandarizacion_genero_bga)

####Estandarizacion de movilidad tanto para el asaltante como para la victima

In [ ]:
# ESTANDARIZACIÓN DE MOVILIDAD

estandarizacion_movilidad_bga = {
    'NO DISPONIBLE': 'NO REPORTADO',
    'A PIE': 'A PIE',
    'CONDUCTOR VEHICULO': 'VEHICULO PARTICULAR',
    'PASAJERO VEHICULO': 'VEHICULO PARTICULAR',
    'CONDUCTOR MOTOCICLETA': 'MOTOCICLETA',
    'PASAJERO MOTOCICLETA': 'MOTOCICLETA',
    'CONDUCTOR BUS': 'TRANSPORTE PUBLICO',
    'PASAJERO BUS': 'TRANSPORTE PUBLICO',
    'CONDUCTOR TAXI': 'TAXI',
    'PASAJERO TAXI': 'TAXI',
    'BICICLETA': 'BICICLETA',
    'PASAJERO METRO': 'METRO',
    'PASAJERO BARCO': 'BARCO',
    'PASAJERO AERONAVE': 'AERONAVE',
}

In [ ]:
df_bucaramanga['MOVIL VICTIMA'] = df_bucaramanga['MOVIL VICTIMA'].replace(estandarizacion_movilidad_bga)
df_bucaramanga['MOVIL AGRESOR'] = df_bucaramanga['MOVIL AGRESOR'].replace(estandarizacion_movilidad_bga)

####Estandarizacion de los delitos de bucaramanga

In [ ]:
# DELITOS SEXUALES
estandarizacion_delitos_sexuales_bga = {
    # Acceso carnal con menores
    'ARTÍCULO 208. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS': 'ACCESO CARNAL CON MENOR',
    'ARTÍCULO 211. ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS (CIRCUNSTANCIAS AGRAVACIÓN)': 'ACCESO CARNAL CON MENOR',

    # Actos sexuales con menores
    'ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS': 'ACTOS SEXUALES CON MENOR',
    'ARTÍCULO 211. ACTOS SEXUALES CON MENOR DE 14 AÑOS (CIRCUNSTANCIAS DE AGRAVACIÓN)': 'ACTOS SEXUALES CON MENOR',

    # Acceso carnal violento
    'ARTÍCULO 205. ACCESO CARNAL VIOLENTO': 'ACCESO CARNAL VIOLENTO',
    'ARTÍCULO 211. ACCESO CARNAL VIOLENTO (CIRCUNSTANCIAS AGRAVACIÓN)': 'ACCESO CARNAL VIOLENTO',

    # Acto sexual violento
    'ARTÍCULO 206. ACTO SEXUAL VIOLENTO': 'ACTO SEXUAL VIOLENTO',
    'ARTÍCULO 211. ACTO SEXUAL VIOLENTO (CIRCUNSTANCIAS DE AGRAVACIÓN)': 'ACTO SEXUAL VIOLENTO',

    # Acceso/acto con incapaz
    'ARTÍCULO 210. ACCESO CARNAL O ACTO SEXUAL ABUSIVO CON INCAPAZ DE RESISTIR': 'ABUSO A PERSONA INCAPAZ',
    'ARTÍCULO 207. ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR': 'ABUSO A PERSONA INCAPAZ',
    'ARTÍCULO 211. ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR  (CIRCUNSTANCIAS AGRAVACIÓN)': 'ABUSO A PERSONA INCAPAZ',
    'ARTÍCULO 211. ACCESO CARNAL O ACTO SEXUAL ABUSIVO CON INCAPAZ DE RESISTIR (CIRCUNSTANCIAS AGRAVACIÓN)': 'ABUSO A PERSONA INCAPAZ',

    # Acoso sexual
    'ARTÍCULO 210 A. ACOSO SEXUAL': 'ACOSO SEXUAL',

    # Pornografía y explotación de menores
    'ARTÍCULO 218. PORNOGRAFÍA CON MENORES': 'EXPLOTACION SEXUAL MENORES',
    'ARTÍCULO 219 A. UTILIZACIÓN O FACILITACIÓN DE MEDIOS DE COMUNICACIÓN PARA OFRECER SERVICIOS SEXUALES DE MENORES': 'EXPLOTACION SEXUAL MENORES',
    'ARTÍCULO 217 A. DEMANDA DE EXPLOTACION SEXUAL COMERCIAL DE PERSONA MENOR DE 18 AÑOS DE EDAD': 'EXPLOTACION SEXUAL MENORES',
    'ARTÍCULO 217. ESTÍMULO A LA PROSTITUCIÓN DE MENORES': 'EXPLOTACION SEXUAL MENORES',
    'ARTÍCULO 213 A. PROXENETISMO CON MENOR DE EDAD': 'EXPLOTACION SEXUAL MENORES',

    # Prostitución forzada
    'ARTÍCULO 213. INDUCCIÓN A LA PROSTITUCIÓN': 'PROSTITUCION FORZADA',
    'ARTÍCULO 214. CONSTREÑIMIENTO A LA PROSTITUCIÓN': 'PROSTITUCION FORZADA',

    # Omisión de denuncia
    'ARTÍCULO 219 B. OMISIÓN DE DENUNCIA': 'OMISION DE DENUNCIA',
}


# HURTOS
estandarizacion_hurtos_bga = {
    'ARTÍCULO 239. HURTO PERSONAS': 'HURTO A PERSONAS',
    'ARTÍCULO 239. HURTO AUTOMOTORES': 'HURTO DE AUTOMOTORES',
    'ARTÍCULO 239. HURTO MOTOCICLETAS': 'HURTO DE MOTOCICLETAS',
    'ARTÍCULO 239. HURTO RESIDENCIAS': 'HURTO A RESIDENCIAS',
    'ARTÍCULO 239. HURTO ENTIDADES COMERCIALES': 'HURTO A COMERCIOS',
}

# LESIONES
estandarizacion_lesiones_bga = {
    'ARTÍCULO 111. LESIONES PERSONALES': 'LESIONES PERSONALES',
    'ARTÍCULO 119. LESIONES PERSONALES ( CIRCUNSTANCIAS DE AGRAVACIÓN)': 'LESIONES PERSONALES AGRAVADAS',
    'ARTÍCULO 120. LESIONES CULPOSAS': 'LESIONES CULPOSAS',
    'ARTÍCULO 120. LESIONES CULPOSAS ( EN ACCIDENTE DE TRANSITO )': 'LESIONES EN TRANSITO',
    'LESION ACCIDENTAL EN TRANSITO': 'LESIONES EN TRANSITO',
    'ARTÍCULO 125. LESIONES AL FETO': 'LESIONES AL FETO',
}

# HOMICIDIOS
estandarizacion_homicidios_bga = {
    'ARTÍCULO 103. HOMICIDIO': 'HOMICIDIO',
    'ARTÍCULO 104A. FEMINICIDIO': 'FEMINICIDIO',
    'ARTÍCULO 109. HOMICIDIO CULPOSO ( EN ACCIDENTE DE TRÁNSITO)': 'HOMICIDIO EN TRANSITO',
    'MUERTE EN ACCIDENTE DE TRANSITO': 'HOMICIDIO EN TRANSITO',
}

#VIOLENCIA
estandarizacion_violencia_bga = {
    'ARTÍCULO 229. VIOLENCIA INTRAFAMILIAR': 'VIOLENCIA INTRAFAMILIAR'
    }

# OTROS DELITOS
estandarizacion_otros_delitos_bga = {
    'ARTÍCULO 244. EXTORSIÓN': 'EXTORSION',
    'ARTÍCULO 347. AMENAZAS': 'AMENAZAS',
    'ARTÍCULO 343. TERRORISMO': 'TERRORISMO',
    'ARTÍCULO 265. DAÑO EN BIEN AJENO': 'DAÑO EN BIEN AJENO',
    'ARTÍCULO 350.  INCENDIO': 'INCENDIO',
    'ARTÍCULO 429. VIOLENCIA CONTRA SERVIDOR PÚBLICO': 'VIOLENCIA CONTRA SERVIDOR PUBLICO',
    'ARTÍCULO 243. ABIGEATO': 'ABIGEATO',
}

In [ ]:


def categorizar_delito_bucaramanga(delito_descripcion):

    #Categoriza el delito basándose en DESCRIPCION CONDUCTA
    #Retorna: (CATEGORIA, TIPO, DETALLADO)

    # Unir todos los diccionarios
    todos_delitos = {
        **estandarizacion_delitos_sexuales_bga,
        **estandarizacion_hurtos_bga,
        **estandarizacion_lesiones_bga,
        **estandarizacion_homicidios_bga,
        **estandarizacion_violencia_bga,
        **estandarizacion_otros_delitos_bga
    }

    # Definir categorías
    if delito_descripcion in estandarizacion_delitos_sexuales_bga:
        categoria = 'DELITOS SEXUALES'
        tipo = estandarizacion_delitos_sexuales_bga[delito_descripcion]

    elif delito_descripcion in estandarizacion_hurtos_bga:
        categoria = 'DELITO CONTRA EL PATRIMONIO'
        tipo = estandarizacion_hurtos_bga[delito_descripcion]

    elif delito_descripcion in estandarizacion_lesiones_bga:
        categoria = 'DELITOS CONTRA LA INTEGRIDAD'
        tipo = estandarizacion_lesiones_bga[delito_descripcion]

    elif delito_descripcion in estandarizacion_homicidios_bga:
        categoria = 'DELITO CONTRA LA VIDA'
        tipo = estandarizacion_homicidios_bga[delito_descripcion]

    elif delito_descripcion in estandarizacion_violencia_bga:
        categoria = 'DELITOS CONTRA LA FAMILIA'
        tipo = estandarizacion_violencia_bga[delito_descripcion]

    elif delito_descripcion in estandarizacion_otros_delitos_bga:
        if any(x in delito_descripcion for x in ['EXTORSIÓN', 'AMENAZAS']):
            categoria = 'DELITOS CONTRA LA LIBERTAD'
        elif any(x in delito_descripcion for x in ['TERRORISMO', 'INCENDIO']):
            categoria = 'DELITOS CONTRA LA SEGURIDAD PUBLICA'
        else:
            categoria = 'OTROS DELITOS'
        tipo = estandarizacion_otros_delitos_bga[delito_descripcion]
    else:
        categoria = 'OTROS DELITOS'
        tipo = 'OTROS'

    detallado = delito_descripcion

    return categoria, tipo, detallado

# Aplicar categorización
df_bucaramanga[['CATEGORIA DELITO', 'TIPO DELITO', 'DELITO DETALLADO']] = df_bucaramanga['DESCRIPCION CONDUCTA'].apply(
    lambda x: pd.Series(categorizar_delito_bucaramanga(x))
)

# Verificar resultados
print("\n=== CATEGORÍAS DE DELITO ===")
print(df_bucaramanga['CATEGORIA DELITO'].value_counts())
print("\n=== TIPOS DE DELITO ===")
print(df_bucaramanga['TIPO DELITO'].value_counts())

print(df_bucaramanga.head())


=== CATEGORÍAS DE DELITO ===
CATEGORIA DELITO
DELITO CONTRA EL PATRIMONIO            74445
DELITOS CONTRA LA INTEGRIDAD           20457
DELITOS CONTRA LA FAMILIA              16601
DELITOS SEXUALES                        4036
DELITOS CONTRA LA LIBERTAD              3988
DELITO CONTRA LA VIDA                   1405
DELITOS CONTRA LA SEGURIDAD PUBLICA        4
OTROS DELITOS                              4
Name: count, dtype: int64

=== TIPOS DE DELITO ===
TIPO DELITO
HURTO A PERSONAS                     56518
LESIONES PERSONALES                  16693
VIOLENCIA INTRAFAMILIAR              16601
HURTO A COMERCIOS                     8764
HURTO DE MOTOCICLETAS                 4769
HURTO A RESIDENCIAS                   4224
LESIONES EN TRANSITO                  3438
AMENAZAS                              3365
ACTOS SEXUALES CON MENOR              1609
HOMICIDIO                             1030
ACCESO CARNAL CON MENOR                715
EXTORSION                              623
ACCESO CARNAL 

In [ ]:
for df in [df_violencia, df_delitos_sexuales, df_hurtos]:
    if 'MUNICIPIO' in df.columns:
        df['MUNICIPIO'] = df['MUNICIPIO'].replace('GAMBITA', 'GÁMBITA')

###Revisamos la informacion de los dataset hasta ahora

In [ ]:
#MOSTRAMOS LA INFORMACION RELEVANTE
print("\n Informacion general:\n" )
df_hurtos.info()
print("\n")
print("\n Valores nulos por columna:")
print(df_hurtos.isnull().sum())
print("\n")
print("\n")
print(f"\n Valores duplicados:")
print(f"{df_hurtos.duplicated().sum()} Valores repetidos")
print("\n Rango de fechas:")
print(f"Fecha mas antigua: {df_hurtos['FECHA HECHO'].min()}")
print(f"Fecha mas reciente: {df_hurtos['FECHA HECHO'].max()}")


 Informacion general:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43870 entries, 0 to 43869
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DEPARTAMENTO      43870 non-null  object
 1   MUNICIPIO         43870 non-null  object
 2   CODIGO DANE       43870 non-null  object
 3   ARMAS MEDIOS      43870 non-null  object
 4   FECHA HECHO       43870 non-null  object
 5   GENERO            39650 non-null  object
 6   GRUPO ETARIO      42859 non-null  object
 7   CANTIDAD          43870 non-null  object
 8   CATEGORIA DELITO  43870 non-null  object
 9   TIPO DELITO       43870 non-null  object
 10  DELITO DETALLADO  43870 non-null  object
dtypes: object(11)
memory usage: 3.7+ MB



 Valores nulos por columna:
DEPARTAMENTO           0
MUNICIPIO              0
CODIGO DANE            0
ARMAS MEDIOS           0
FECHA HECHO            0
GENERO              4220
GRUPO ETARIO        1011
CANTIDAD              

In [ ]:
#MOSTRAMOS LOS DATOS REELEVANTES
print("\n Informacion general:")
df_delitos_sexuales.info()
print("\n")
print("\n Valores nulos por columna:")
print(df_delitos_sexuales.isnull().sum())
print("\n")
print("\n")
print(f"\n Valores duplicados:")
print(f"{df_delitos_sexuales.duplicated().sum()} Valores repetidos")
print("\n Rango de fechas:")
print(f"Fecha mas antigua: {df_delitos_sexuales['FECHA HECHO'].min()}")
print(f"Fecha mas reciente: {df_delitos_sexuales['FECHA HECHO'].max()}")


 Informacion general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382154 entries, 0 to 382153
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   DEPARTAMENTO      382154 non-null  object
 1   MUNICIPIO         382151 non-null  object
 2   CODIGO DANE       382154 non-null  object
 3   ARMAS MEDIOS      382154 non-null  object
 4   FECHA HECHO       382154 non-null  object
 5   GENERO            380124 non-null  object
 6   GRUPO ETARIO      381637 non-null  object
 7   CANTIDAD          382154 non-null  object
 8   CATEGORIA DELITO  382154 non-null  object
 9   TIPO DELITO       382154 non-null  object
 10  DELITO DETALLADO  382154 non-null  object
dtypes: object(11)
memory usage: 32.1+ MB



 Valores nulos por columna:
DEPARTAMENTO           0
MUNICIPIO              3
CODIGO DANE            0
ARMAS MEDIOS           0
FECHA HECHO            0
GENERO              2030
GRUPO ETARIO         517
CANTIDA

In [ ]:
#MOSTRAMOS LOS DATOS REELEVANTES
print("\n Informacion general:")
df_violencia.info()
print("\n")
print("\n Valores nulos por columna:")
print(df_violencia.isnull().sum())
print("\n")
print("\n Valores duplicados")
print(f"{df_violencia.duplicated().sum()} Valores repetidos")
print("\n Rango de fechas:")
print(f"Fecha mas antigua: {df_violencia['FECHA HECHO'].min()}")
print(f"Fecha mas reciente: {df_violencia['FECHA HECHO'].max()}")


 Informacion general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660756 entries, 0 to 660755
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   DEPARTAMENTO      660756 non-null  object
 1   MUNICIPIO         660752 non-null  object
 2   CODIGO DANE       660756 non-null  object
 3   ARMAS MEDIOS      660756 non-null  object
 4   FECHA HECHO       660756 non-null  object
 5   GENERO            658150 non-null  object
 6   GRUPO ETARIO      659732 non-null  object
 7   CANTIDAD          660756 non-null  object
 8   CATEGORIA DELITO  660756 non-null  object
 9   TIPO DELITO       660756 non-null  object
 10  DELITO DETALLADO  660756 non-null  object
dtypes: object(11)
memory usage: 55.5+ MB



 Valores nulos por columna:
DEPARTAMENTO           0
MUNICIPIO              4
CODIGO DANE            0
ARMAS MEDIOS           0
FECHA HECHO            0
GENERO              2606
GRUPO ETARIO        1024
CANTIDA

In [ ]:
print("\n Informacion general:")
df_bucaramanga.info()
print("\n")
print("\n Valores nulos por columna:")
print(df_bucaramanga.isnull().sum())
print("\n")
print("\n Valores duplicados")
print(f"{df_bucaramanga.duplicated().sum()} Valores repetidos")
print("\n Rango de fechas:")
print(f"Fecha mas antigua: {df_bucaramanga['FECHA HECHO'].min()}")
print(f"Fecha mas reciente: {df_bucaramanga['FECHA HECHO'].max()}")


 Informacion general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120940 entries, 0 to 120939
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   DESCRIPCION CONDUCTA  120940 non-null  object
 1   ARMAS MEDIOS          120940 non-null  object
 2   BARRIO                120940 non-null  object
 3   FECHA HECHO           120940 non-null  object
 4   HORA HECHO            120940 non-null  object
 5   EDAD                  120940 non-null  object
 6   GENERO                120940 non-null  object
 7   MOVIL VICTIMA         120940 non-null  object
 8   MOVIL AGRESOR         120940 non-null  object
 9   CLASE SITIO           120940 non-null  object
 10  ARTICULO              120940 non-null  object
 11  DELITO                120940 non-null  object
 12  CURSO VIDA            120940 non-null  object
 13  CURSO VIDA ORDEN      120940 non-null  object
 14  AÑO                   120940 non-null  object

###Verificacion y tratamientos de duplicados y nulos

####Verificacion de duplicados

In [ ]:

print("VERIFICACIÓN DE DUPLICADOS")
print("\n")


def verificar_duplicados(df, nombre):
    print(f"\n{nombre}")
    print("\n")

    for column in df.select_dtypes(include='object').columns:
      print(df[column].unique())

    # Duplicados totales (todas las columnas)
    duplicados_totales = df.duplicated().sum()
    print(f"Duplicados exactos: {duplicados_totales:,}")

    if duplicados_totales > 0:
        print(f"Porcentaje: {(duplicados_totales/len(df)*100):.2f}%")

    # Duplicados por campos clave (sin CANTIDAD)
    columnas_clave = [col for col in df.columns if col != 'CANTIDAD']
    duplicados_clave = df.duplicated(subset=columnas_clave).sum()
    print(f"Duplicados en campos clave (sin CANTIDAD): {duplicados_clave:,}")

    if duplicados_clave > 0:
        print(f"Porcentaje: {(duplicados_clave/len(df)*100):.2f}%")

    return duplicados_totales, duplicados_clave

# Verificar cada dataset
dup_hurtos_total, dup_hurtos_clave = verificar_duplicados(df_hurtos, "HURTOS")
dup_delitos_total, dup_delitos_clave = verificar_duplicados(df_delitos_sexuales, "DELITOS SEXUALES")
dup_violencia_total, dup_violencia_clave = verificar_duplicados(df_violencia, "VIOLENCIA INTRAFAMILIAR")
dup_bucaramanga_total, dup_bucaramanga_clave = verificar_duplicados(df_bucaramanga, "MUNICIPIO DE BUCARAMANGA")

# Resumen
print("\n")
print("RESUMEN")
print("\n")
print(f"Hurtos: {dup_hurtos_total:,} duplicados")
print(f"Delitos Sexuales: {dup_delitos_total:,} duplicados")
print(f"Violencia Intrafamiliar: {dup_violencia_total:,} duplicados")
print(f"MUNICIPIO DE BUCARAMANGA: {dup_bucaramanga_total:,} duplicados")

VERIFICACIÓN DE DUPLICADOS



HURTOS


['GUAJIRA' 'ANTIOQUIA' 'CUNDINAMARCA' 'MAGDALENA' 'CESAR' 'META'
 'ATLÁNTICO' 'GUAVIARE' 'CASANARE' 'SUCRE' 'CÓRDOBA' 'RISARALDA' 'HUILA'
 'NORTE DE SANTANDER' 'VICHADA' 'CALDAS' 'CAQUETÁ' 'TOLIMA' 'BOYACÁ'
 'SANTANDER' 'VALLE' 'CAUCA' 'ARAUCA' 'NARIÑO' 'PUTUMAYO' 'BOLÍVAR'
 'GUAINÍA' 'QUINDÍO' 'CHOCÓ' 'AMAZONAS' 'SAN ANDRÉS' 'VAUPÉS']
['LA JAGUA DEL PILAR' 'ANDES' 'SAN JUAN DEL CESAR' 'FUSAGASUGÁ'
 'ARACATACA' 'AGUSTÍN CODAZZI' 'BARRANCA DE UPÍA' 'MANATÍ'
 'SAN JOSÉ DEL GUAVIARE ' 'MONTERREY' 'SAN ALBERTO' 'FUNDACIÓN'
 'SAN MARCOS' 'SANTA BÁRBARA DE PINTO' 'PLANETA RICA' 'MISTRATÓ' 'YOPAL '
 'CHINÚ' 'LORICA' 'TÁMARA' 'PAICOL' 'SAN JOSÉ DE URÉ' 'MONTELÍBANO'
 'REMEDIOS' 'BUENAVISTA' 'SANTA ROSA DE OSOS' 'CAUCASIA' 'RIONEGRO'
 'CHIGORODÓ' 'CAROLINA' 'URRAO' 'ALBÁN' 'EL CARMEN DE VIBORAL' 'LA CEJA'
 'SARDINATA' 'SAN ANTERO' 'BETULIA' 'CIUDAD BOLÍVAR' 'SAHAGÚN'
 'PUEBLO NUEVO' 'SANTA ROSALÍA' 'ANSERMA' 'MOMIL' 'SONSON' 'FLORENCIA '
 'VILLAVICENCIO ' 

####Tratamiento de duplicado

In [ ]:
print("ELIMINACIÓN DE DUPLICADOS \n")


# REGISTROS ANTES
print("\nRegistros antes de limpieza:")
print(f"Hurtos: {len(df_hurtos):,}")
print(f"Delitos Sexuales: {len(df_delitos_sexuales):,}")
print(f"Violencia Intrafamiliar: {len(df_violencia):,}")
print(f"Municipio de bucaramanga: {len(df_bucaramanga):,}")
print(f"Duplicados totales {(dup_hurtos_total+dup_violencia_total+dup_delitos_total+dup_bucaramanga_total):,}")

# ELIMINAR DUPLICADOS
df_hurtos_clean = df_hurtos.drop_duplicates()
df_bucaramanga_clean = df_bucaramanga.drop_duplicates()
df_delitos_sexuales_clean = df_delitos_sexuales.drop_duplicates()
df_violencia_clean = df_violencia.copy()  # No tiene duplicados

# REGISTROS DESPUES
print("\nRegistros después de limpieza:")
print(f"Hurtos: {len(df_hurtos_clean):,} (eliminados: {len(df_hurtos) - len(df_hurtos_clean):,})")
print(f"Bucaramanga: {len(df_bucaramanga_clean):,} (eliminados: {len(df_bucaramanga) - len(df_bucaramanga_clean):,})")
print(f"Delitos Sexuales: {len(df_delitos_sexuales_clean):,} (eliminados: {len(df_delitos_sexuales) - len(df_delitos_sexuales_clean):,})")
print(f"Violencia Intrafamiliar: {len(df_violencia_clean):,} (sin cambios)")

print("\n")
print(f"Total registros limpios: {len(df_hurtos_clean) + len(df_delitos_sexuales_clean) + len(df_violencia_clean)+ len(df_bucaramanga_clean):,}")

ELIMINACIÓN DE DUPLICADOS 


Registros antes de limpieza:
Hurtos: 43,870
Delitos Sexuales: 382,154
Violencia Intrafamiliar: 660,756
Municipio de bucaramanga: 120,940
Duplicados totales 50,212

Registros después de limpieza:
Hurtos: 43,005 (eliminados: 865)
Bucaramanga: 120,704 (eliminados: 236)
Delitos Sexuales: 333,214 (eliminados: 48,940)
Violencia Intrafamiliar: 660,756 (sin cambios)


Total registros limpios: 1,157,679


####Verificacion de nulos

In [ ]:
#Verificacion de nulos
print("VERIFICACIÓN DE VALORES NULOS")
print("\n")

print(df_hurtos_clean.isnull().sum())
print("\n")
print(df_delitos_sexuales_clean.isnull().sum())
print("\n")
print(df_violencia_clean.isnull().sum())
print("\n")
print(df_bucaramanga_clean.isnull().sum())

VERIFICACIÓN DE VALORES NULOS


DEPARTAMENTO           0
MUNICIPIO              0
CODIGO DANE            0
ARMAS MEDIOS           0
FECHA HECHO            0
GENERO              4141
GRUPO ETARIO        1002
CANTIDAD               0
CATEGORIA DELITO       0
TIPO DELITO            0
DELITO DETALLADO       0
dtype: int64


DEPARTAMENTO           0
MUNICIPIO              3
CODIGO DANE            0
ARMAS MEDIOS           0
FECHA HECHO            0
GENERO              1958
GRUPO ETARIO         501
CANTIDAD               0
CATEGORIA DELITO       0
TIPO DELITO            0
DELITO DETALLADO       0
dtype: int64


DEPARTAMENTO           0
MUNICIPIO              4
CODIGO DANE            0
ARMAS MEDIOS           0
FECHA HECHO            0
GENERO              2606
GRUPO ETARIO        1024
CANTIDAD               0
CATEGORIA DELITO       0
TIPO DELITO            0
DELITO DETALLADO       0
dtype: int64


DESCRIPCION CONDUCTA    0
ARMAS MEDIOS            0
BARRIO                  0
FECHA HECHO         

####Tratamiento de nulos

In [ ]:
#TRATAMIENTO DE VALORES NULOS
print("TRATAMIENTO DE VALORES NULOS - VIOLENCIA INTRAFAMILIAR")

# VALORES NULOS ANTES
print("\nValores nulos antes:")
print(df_violencia_clean.isnull().sum())

# Estrategia:
# - MUNICIPIO: eliminar (son solo 4 registros)
# - GENERO y GRUPO ETARIO: marcar como 'NO REPORTADO'

# ELIMINAR REGISTROS SIN MUNICIPIOS
df_violencia_clean = df_violencia_clean.dropna(subset=['MUNICIPIO'])

# RELLENAR GENERO Y GRUPO ETARIO
df_violencia_clean['GENERO'] = df_violencia_clean['GENERO'].fillna('NO REPORTADO')
df_violencia_clean['GRUPO ETARIO'] = df_violencia_clean['GRUPO ETARIO'].fillna('NO REPORTADO')

print("\nValores nulos después:")
print(df_violencia_clean.isnull().sum())

print(f"\nRegistros finales Violencia Intrafamiliar: {len(df_violencia_clean):,}")
print(f"Registros eliminados por falta de municipio: 4")


TRATAMIENTO DE VALORES NULOS - VIOLENCIA INTRAFAMILIAR

Valores nulos antes:
DEPARTAMENTO           0
MUNICIPIO              4
CODIGO DANE            0
ARMAS MEDIOS           0
FECHA HECHO            0
GENERO              2606
GRUPO ETARIO        1024
CANTIDAD               0
CATEGORIA DELITO       0
TIPO DELITO            0
DELITO DETALLADO       0
dtype: int64

Valores nulos después:
DEPARTAMENTO        0
MUNICIPIO           0
CODIGO DANE         0
ARMAS MEDIOS        0
FECHA HECHO         0
GENERO              0
GRUPO ETARIO        0
CANTIDAD            0
CATEGORIA DELITO    0
TIPO DELITO         0
DELITO DETALLADO    0
dtype: int64

Registros finales Violencia Intrafamiliar: 660,752
Registros eliminados por falta de municipio: 4


###Tratamiento de fechas

####Conversion de fechas

Realizamos la conversion de los datos de tipo objeto a tipo fecha

In [ ]:
# CONVERSIÓN DE FECHAS
print("CONVERSIÓN DE FECHAS A DATETIME")
print("\n")

# CONVERTIR FECHAS (FORMATO DD/MM/YYYY)
print("\nConvirtiendo fechas...")

df_hurtos_clean['FECHA HECHO'] = pd.to_datetime(df_hurtos_clean['FECHA HECHO'], format='%d/%m/%Y')
df_delitos_sexuales_clean['FECHA HECHO'] = pd.to_datetime(df_delitos_sexuales_clean['FECHA HECHO'], format='%d/%m/%Y')
df_violencia_clean['FECHA HECHO'] = pd.to_datetime(df_violencia_clean['FECHA HECHO'], format='%d/%m/%Y')
df_bucaramanga_clean['FECHA HECHO'] = pd.to_datetime(df_bucaramanga_clean['FECHA HECHO'], format='%Y-%m-%dT%H:%M:%S.000')

print("Conversión completada")

# VERIFICAR RANGOS
print("\nRangos de fechas:")
print(f"Hurtos: {df_hurtos_clean['FECHA HECHO'].min()} a {df_hurtos_clean['FECHA HECHO'].max()}")
print(f"Delitos Sexuales: {df_delitos_sexuales_clean['FECHA HECHO'].min()} a {df_delitos_sexuales_clean['FECHA HECHO'].max()}")
print(f"Violencia Intrafamiliar: {df_violencia_clean['FECHA HECHO'].min()} a {df_violencia_clean['FECHA HECHO'].max()}")
print(f"Bucaramanga: {df_bucaramanga_clean["FECHA HECHO"].min()} a {df_bucaramanga_clean["FECHA HECHO"].max()}")

CONVERSIÓN DE FECHAS A DATETIME



Convirtiendo fechas...
Conversión completada

Rangos de fechas:
Hurtos: 2010-01-01 00:00:00 a 2025-10-29 00:00:00
Delitos Sexuales: 2010-01-01 00:00:00 a 2025-10-31 00:00:00
Violencia Intrafamiliar: 2010-01-01 00:00:00 a 2025-10-31 00:00:00
Bucaramanga: 2016-01-01 00:00:00 a 2025-05-31 00:00:00


####Visualizacion del rango de **fechas**

Visualizamos los rango de fechas a utilizar en los 3 dataset de la policia nacional

In [ ]:
# FECHE LIMITE (CUANDO SE GENERO EL REPORTE)
fecha_limite = pd.to_datetime('2025-05-30')

print(f"\nFecha del reporte: {fecha_limite.date()}")
print(f" 2025-05-30")

# CONTAR REGISTROS CON FECHAS POSTERIOR AL REPORTE
print("\nRegistros con fechas posteriores al reporte:")
print(f"Hurtos: {(df_hurtos_clean['FECHA HECHO'] > fecha_limite).sum():,}")
print(f"Delitos Sexuales: {(df_delitos_sexuales_clean['FECHA HECHO'] > fecha_limite).sum():,}")
print(f"Violencia Intrafamiliar: {(df_violencia_clean['FECHA HECHO'] > fecha_limite).sum():,}")

# VER DISTRIBUCION POR YEAR
print("\nDistribución por año - Hurtos:")
print(df_hurtos_clean['FECHA HECHO'].dt.year.value_counts().sort_index())

# VER DISTRIBUCION POR YEAR
print("\nDistribución por año - delitos sexuales:")
print(df_delitos_sexuales_clean['FECHA HECHO'].dt.year.value_counts().sort_index())

# VER DISTRIBUCION POR YEAR
print("\nDistribución por año - Violencia intrafamiliar:")
print(df_violencia_clean['FECHA HECHO'].dt.year.value_counts().sort_index())


Fecha del reporte: 2025-05-30
 2025-05-30

Registros con fechas posteriores al reporte:
Hurtos: 279
Delitos Sexuales: 11,988
Violencia Intrafamiliar: 24,987

Distribución por año - Hurtos:
FECHA HECHO
2010    1814
2011    1851
2012    2371
2013    2814
2014    2330
2015    2521
2016    3392
2017    5591
2018    4414
2019    3729
2020    2750
2021    2598
2022    2818
2023    1796
2024    1436
2025     780
Name: count, dtype: int64

Distribución por año - delitos sexuales:
FECHA HECHO
2010     6463
2011     7711
2012    10784
2013    11123
2014    12112
2015    20273
2016    21941
2017    25436
2018    32141
2019    32607
2020    27325
2021    29765
2022    23392
2023    23558
2024    25234
2025    23349
Name: count, dtype: int64

Distribución por año - Violencia intrafamiliar:
FECHA HECHO
2010    14992
2011    16925
2012    20240
2013    21792
2014    29760
2015    44501
2016    49191
2017    49493
2018    46363
2019    50689
2020    53687
2021    57864
2022    49982
2023    50850
202

####Filtrado de las fechas a usar

Realizamos el filtrado de las fechas que tienen los datasets en comun

In [ ]:
print("FILTRADO A PERIODO COMÚN: 2010-2024")


# DEFINIR PERIODO
fecha_inicio = pd.to_datetime('2010-01-01')
fecha_fin = pd.to_datetime('2025-05-30')

print(f"\nPeriodo seleccionado: {fecha_inicio.date()} a {fecha_fin.date()}")

# REGISTROS TOMADOS ANTES DEL FITRADO
print("\nRegistros antes del filtro:")
print(f"Hurtos: {len(df_hurtos_clean):,}")
print(f"Delitos Sexuales: {len(df_delitos_sexuales_clean):,}")
print(f"Violencia Intrafamiliar: {len(df_violencia_clean):,}")

# APLICAR FILTRO
df_hurtos_final = df_hurtos_clean[
    (df_hurtos_clean['FECHA HECHO'] >= fecha_inicio) &
    (df_hurtos_clean['FECHA HECHO'] <= fecha_fin)
].copy()

df_delitos_sexuales_final = df_delitos_sexuales_clean[
    (df_delitos_sexuales_clean['FECHA HECHO'] >= fecha_inicio) &
    (df_delitos_sexuales_clean['FECHA HECHO'] <= fecha_fin)
].copy()

df_violencia_final = df_violencia_clean[
    (df_violencia_clean['FECHA HECHO'] >= fecha_inicio) &
    (df_violencia_clean['FECHA HECHO'] <= fecha_fin)
].copy()

# REGISTROS TOTALES DESPUES DEL FILTRADO
print("\nRegistros después del filtro:")
print(f"Hurtos: {len(df_hurtos_final):,} (eliminados: {len(df_hurtos_clean) - len(df_hurtos_final):,})")
print(f"Delitos Sexuales: {len(df_delitos_sexuales_final):,} (eliminados: {len(df_delitos_sexuales_clean) - len(df_delitos_sexuales_final):,})")
print(f"Violencia Intrafamiliar: {len(df_violencia_final):,} (eliminados: {len(df_violencia_clean) - len(df_violencia_final):,})")

print("\n")
print(f"TOTAL FINAL: {len(df_hurtos_final) + len(df_delitos_sexuales_final) + len(df_violencia_final):,} registros")
print("Periodo: 15 años (2010-2025)")

FILTRADO A PERIODO COMÚN: 2010-2024

Periodo seleccionado: 2010-01-01 a 2025-05-30

Registros antes del filtro:
Hurtos: 43,005
Delitos Sexuales: 333,214
Violencia Intrafamiliar: 660,752

Registros después del filtro:
Hurtos: 42,726 (eliminados: 279)
Delitos Sexuales: 321,226 (eliminados: 11,988)
Violencia Intrafamiliar: 635,765 (eliminados: 24,987)


TOTAL FINAL: 999,717 registros
Periodo: 15 años (2010-2025)


In [ ]:
# @title
print("CREACIÓN DE CÓDIGO DE MUNICIPIO")
print("=" * 60)

# Función simplificada para código de 5 dígitos
def extraer_cod_municipio(codigo_dane):
    try:
        codigo_str = str(int(float(str(codigo_dane).replace(',', '.'))))
    except ValueError:
        codigo_str = str(codigo_dane)

    # Tomar primeros 5 dígitos after potential conversion and cleaning
    if len(codigo_str) > 5:
        cod_limpio = codigo_str[:-3]
    else:
        cod_limpio = codigo_str

    # Rellenar a 5 dígitos con ceros a la izquierda
    return cod_limpio.zfill(5)


# Aplicar a los 3 datasets de delitos
print("\nCreando columna COD_MUNICIPIO en datasets de delitos...")

# Create a NEW column 'COD_MUNICIPIO' using the existing 'CODIGO DANE'
# Do not overwrite 'CODIGO DANE' as it might be needed for other reasons
df_hurtos_final['COD_MUNICIPIO'] = df_hurtos_final['CODIGO DANE'].apply(extraer_cod_municipio)
df_delitos_sexuales_final['COD_MUNICIPIO'] = df_delitos_sexuales_final['CODIGO DANE'].apply(extraer_cod_municipio)
df_violencia_final['COD_MUNICIPIO'] = df_violencia_final['CODIGO DANE'].apply(extraer_cod_municipio)

print("✓ Códigos creados")

# Verificación
print("\n Verificación de códigos:")

print("\nHurtos - ejemplos:")
# Use the newly created 'COD_MUNICIPIO'
print(df_hurtos_final[['CODIGO DANE', 'COD_MUNICIPIO', 'MUNICIPIO']].head(10))

print("\nDelitos Sexuales - ejemplos:")
# Use the newly created 'COD_MUNICIPIO'
print(df_delitos_sexuales_final[['CODIGO DANE', 'COD_MUNICIPIO', 'MUNICIPIO']].head(10))


# Verificar longitud de códigos
print("\n Longitud de códigos:")
print(f"Hurtos: {df_hurtos_final['COD_MUNICIPIO'].str.len().unique()}")
print(f"Delitos: {df_delitos_sexuales_final['COD_MUNICIPIO'].str.len().unique()}")
print(f"Violencia: {df_violencia_final['COD_MUNICIPIO'].str.len().unique()}")

# Verificar coincidencias
codigos_hurtos_mpio = set(df_hurtos_final['COD_MUNICIPIO'].unique())
codigos_delitos_mpio = set(df_delitos_sexuales_final['COD_MUNICIPIO'].unique())
codigos_violencia_mpio = set(df_violencia_final['COD_MUNICIPIO'].unique())


print("\n" + "=" * 60)
print("VERIFICACIÓN DE COINCIDENCIAS (NIVEL MUNICIPIO):")
print("=" * 60)
print(f"Hurtos - municipios únicos: {len(codigos_hurtos_mpio)}")
print(f"Delitos - municipios únicos: {len(codigos_delitos_mpio)}")
print(f"Violencia - municipios únicos: {len(codigos_violencia_mpio)}")

CREACIÓN DE CÓDIGO DE MUNICIPIO

Creando columna COD_MUNICIPIO en datasets de delitos...
✓ Códigos creados

 Verificación de códigos:

Hurtos - ejemplos:
  CODIGO DANE COD_MUNICIPIO               MUNICIPIO
0    44420000         44420      LA JAGUA DEL PILAR
1    05034007         05034                   ANDES
2    44650000         44650      SAN JUAN DEL CESAR
3    25290000         25290              FUSAGASUGÁ
4    47053000         47053               ARACATACA
5    20013000         20013         AGUSTÍN CODAZZI
6    50110000         50110        BARRANCA DE UPÍA
7    08436000         08436                  MANATÍ
8    08436000         08436                  MANATÍ
9    95001000         95001  SAN JOSÉ DEL GUAVIARE 

Delitos Sexuales - ejemplos:
   CODIGO DANE COD_MUNICIPIO            MUNICIPIO
0     86001000         86001               MOCOA 
1     85010000         85010              AGUAZUL
2     85010000         85010              AGUAZUL
3     76520000         76520              PA

### Creacion de caracteristicas

In [ ]:
print("CREACIÓN DE VARIABLES TEMPORALES EN ESPAÑOL")
print("=" * 60)

# Diccionarios de traducción
DIAS_ES = {
    'Monday': 'LUNES',
    'Tuesday': 'MARTES',
    'Wednesday': 'MIERCOLES',
    'Thursday': 'JUEVES',
    'Friday': 'VIERNES',
    'Saturday': 'SABADO',
    'Sunday': 'DOMINGO'
}

MESES_ES = {
    'January': 'ENERO',
    'February': 'FEBRERO',
    'March': 'MARZO',
    'April': 'ABRIL',
    'May': 'MAYO',
    'June': 'JUNIO',
    'July': 'JULIO',
    'August': 'AGOSTO',
    'September': 'SEPTIEMBRE',
    'October': 'OCTUBRE',
    'November': 'NOVIEMBRE',
    'December': 'DICIEMBRE'
}

def crear_features_temporales(df, nombre_dataset):
    """
    Crea variables temporales a partir de FECHA HECHO
    """
    print(f"\nProcesando: {nombre_dataset}")

    # Extraer componentes de fecha
    df['AÑO'] = df['FECHA HECHO'].dt.year
    df['MES'] = df['FECHA HECHO'].dt.month
    df['DIA_SEMANA'] = df['FECHA HECHO'].dt.dayofweek  # 0=Lunes, 6=Domingo

    # Nombres en inglés primero
    df['NOMBRE_DIA_EN'] = df['FECHA HECHO'].dt.day_name()
    df['NOMBRE_MES_EN'] = df['FECHA HECHO'].dt.month_name()

    # Traducir al español
    df['NOMBRE_DIA'] = df['NOMBRE_DIA_EN'].map(DIAS_ES)
    df['NOMBRE_MES'] = df['NOMBRE_MES_EN'].map(MESES_ES)

    # Eliminar columnas en inglés
    df = df.drop(['NOMBRE_DIA_EN', 'NOMBRE_MES_EN'], axis=1)

    # Otras variables temporales
    df['TRIMESTRE'] = df['FECHA HECHO'].dt.quarter
    df['SEMESTRE'] = df['FECHA HECHO'].dt.month.apply(lambda x: 1 if x <= 6 else 2)

    print(f"  ✓ Variables temporales creadas en español")

    return df

# Aplicar a los 3 datasets
df_hurtos_geo = crear_features_temporales(df_hurtos_final, "Hurtos")
df_delitos_geo = crear_features_temporales(df_delitos_sexuales_final, "Delitos Sexuales")
df_violencia_geo = crear_features_temporales(df_violencia_final, "Violencia Intrafamiliar")

print("\n" + "=" * 60)
print("VERIFICACIÓN DE NUEVAS COLUMNAS EN ESPAÑOL")
print("=" * 60)

# Mostrar ejemplo de las nuevas variables
print("\n📅 Ejemplo - Hurtos:")
ejemplo = df_hurtos_geo[['FECHA HECHO', 'AÑO', 'MES', 'NOMBRE_MES', 'DIA_SEMANA', 'NOMBRE_DIA', 'TRIMESTRE', 'SEMESTRE']].head(10)
print(ejemplo.to_string(index=False))

print("\n📊 Verificación de valores únicos:")
print(f"\nDías de la semana:")
print(sorted(df_hurtos_geo['NOMBRE_DIA'].unique()))

print(f"\nMeses:")
print(sorted(df_hurtos_geo['NOMBRE_MES'].unique(), key=lambda x: list(MESES_ES.values()).index(x)))

print("\n📋 Columnas totales ahora:")
print(f"Hurtos: {len(df_hurtos_geo.columns)} columnas")
print(f"Delitos: {len(df_delitos_geo.columns)} columnas")
print(f"Violencia: {len(df_violencia_geo.columns)} columnas")

print("\n✅ Variables temporales en ESPAÑOL creadas correctamente")

CREACIÓN DE VARIABLES TEMPORALES EN ESPAÑOL

Procesando: Hurtos
  ✓ Variables temporales creadas en español

Procesando: Delitos Sexuales
  ✓ Variables temporales creadas en español

Procesando: Violencia Intrafamiliar
  ✓ Variables temporales creadas en español

VERIFICACIÓN DE NUEVAS COLUMNAS EN ESPAÑOL

📅 Ejemplo - Hurtos:
FECHA HECHO  AÑO  MES NOMBRE_MES  DIA_SEMANA NOMBRE_DIA  TRIMESTRE  SEMESTRE
 2024-04-28 2024    4      ABRIL           6    DOMINGO          2         1
 2020-02-18 2020    2    FEBRERO           1     MARTES          1         1
 2020-05-02 2020    5       MAYO           5     SABADO          2         1
 2019-04-05 2019    4      ABRIL           4    VIERNES          2         1
 2017-08-07 2017    8     AGOSTO           0      LUNES          3         2
 2017-08-19 2017    8     AGOSTO           5     SABADO          3         2
 2017-08-27 2017    8     AGOSTO           6    DOMINGO          3         2
 2016-11-29 2016   11  NOVIEMBRE           1     MARTES 

###Combinacion del Dataset

In [ ]:

print("=" * 60)
print("\nCombinando datasets...")

df_completo_s = pd.concat([
    df_hurtos_geo.assign(TIPO_DELITO='HURTO'),
    df_delitos_geo.assign(TIPO_DELITO='DELITO_SEXUAL'),
    df_violencia_geo.assign(TIPO_DELITO='VIOLENCIA_INTRAFAMILIAR')
], ignore_index=True)

# Convertir CANTIDAD a numérico después de la concatenación
df_completo_s['CANTIDAD'] = pd.to_numeric(df_completo_s['CANTIDAD'], errors='coerce')


Combinando datasets...


####Filtrado por departamento (santander)

In [ ]:
# Filtrar solo Santander
print("\n Filtrando Santander...")

df_completo = df_completo_s[df_completo_s['DEPARTAMENTO'] == 'SANTANDER'].copy()


 Filtrando Santander...


####Visualizamos el filtrado

In [ ]:
print(f"\n Dataset filtrado")
print(f"   Registros totales: {len(df_completo_s):,}")
print(f"   Registros Santander: {len(df_completo):,}")
print(f"   Porcentaje: {len(df_completo)/len(df_completo_s)*100:.2f}%")

# Verificar municipios de Santander
print(f"\n Municipios de Santander: {df_completo['MUNICIPIO'].nunique()}")
print(f"Dataset completo: {len(df_completo):,} registros")



 Dataset filtrado
   Registros totales: 999,717
   Registros Santander: 73,255
   Porcentaje: 7.33%

 Municipios de Santander: 87
Dataset completo: 73,255 registros


####Unimos la tabla zonas con el dataframe

In [ ]:

df_zonas = pd.DataFrame(provincias_municipios, columns=['ZONA', 'MUNICIPIO'])

print("=== DF_ZONAS (Convertido) ===")
print(f"Total municipios: {len(df_zonas)}")
print(f"Zonas únicas: {df_zonas['ZONA'].nunique()}")
print(f"\nDistribución por zona:")
print(df_zonas['ZONA'].value_counts())


print("\n\n=== MATCH ENTRE DATASETS ===")
municipios_delitos = set(df_completo['MUNICIPIO'].str.upper().unique())
municipios_catalogo = set(df_zonas['MUNICIPIO'].str.upper().unique())

print(f"Municipios en delitos: {len(municipios_delitos)}")
print(f"Municipios en catálogo: {len(municipios_catalogo)}")
print(f"Municipios en ambos: {len(municipios_delitos & municipios_catalogo)}")

print(f"\nMunicipios en delitos pero NO en catálogo:")
print(sorted(municipios_delitos - municipios_catalogo))

print(f"\nMunicipios en catálogo pero NO en delitos:")
print(sorted(municipios_catalogo - municipios_delitos))

print("\n\n=== PREVIEW MERGE ===")
# Normalizar nombres para el merge
df_completo['MUNICIPIO_NORM'] = df_completo['MUNICIPIO'].str.upper().str.strip()
df_zonas['MUNICIPIO_NORM'] = df_zonas['MUNICIPIO'].str.upper().str.strip()

# Hacer el merge
df_con_zona = df_completo.merge(df_zonas[['MUNICIPIO_NORM', 'ZONA']],
                                 on='MUNICIPIO_NORM',
                                 how='left',
                                 suffixes=('_original', '_catalogo'))

print(f"Registros después del merge: {len(df_con_zona)}")
print(f"Registros CON zona asignada: {df_con_zona['ZONA'].notna().sum()}")
print(f"Registros SIN zona asignada: {df_con_zona['ZONA'].isna().sum()}")

print("\n\nDistribución de delitos por zona (catálogo):")
print(df_con_zona['ZONA'].value_counts())

=== DF_ZONAS (Convertido) ===
Total municipios: 87
Zonas únicas: 7

Distribución por zona:
ZONA
Vélez            19
Guanentá         17
Comunera         16
García-Rovira    13
Metropolitana    10
Soto Norte        6
Yariguíes         6
Name: count, dtype: int64


=== MATCH ENTRE DATASETS ===
Municipios en delitos: 87
Municipios en catálogo: 87
Municipios en ambos: 86

Municipios en delitos pero NO en catálogo:
['BUCARAMANGA ']

Municipios en catálogo pero NO en delitos:
['BUCARAMANGA']


=== PREVIEW MERGE ===
Registros después del merge: 73255
Registros CON zona asignada: 73255
Registros SIN zona asignada: 0


Distribución de delitos por zona (catálogo):
ZONA
Metropolitana    43322
Yariguíes        14138
Vélez             5772
Guanentá          4601
Comunera          3431
García-Rovira     1409
Soto Norte         582
Name: count, dtype: int64


##Modelado del departamento santander

In [ ]:
print("=" * 80)
print("MODELO ZONA + TIPO DELITO - VERSIÓN CORREGIDA (SIN FUGA)")
print("=" * 80)

# ============================================================
# PASO 1: CREAR DATASET SEMANAL
# ============================================================

print("\n[PASO 1] Creando dataset agregado...")

df_con_zona['FECHA_HECHO'] = pd.to_datetime(df_con_zona['FECHA HECHO'])
df_con_zona['AÑO_SEMANA'] = df_con_zona['FECHA_HECHO'].dt.strftime('%Y-W%V')
df_con_zona['FECHA_INICIO_SEMANA'] = df_con_zona['FECHA_HECHO'] - pd.to_timedelta(
    df_con_zona['FECHA_HECHO'].dt.dayofweek, unit='d'
)

df_semanal_tipo = df_con_zona.groupby(
    ['ZONA', 'TIPO_DELITO', 'AÑO_SEMANA', 'FECHA_INICIO_SEMANA']
).agg({
    'CANTIDAD': 'sum'
}).reset_index()

df_semanal_tipo.rename(columns={'CANTIDAD': 'TOTAL_DELITOS'}, inplace=True)

df_semanal_tipo['AÑO'] = df_semanal_tipo['FECHA_INICIO_SEMANA'].dt.year
df_semanal_tipo['MES'] = df_semanal_tipo['FECHA_INICIO_SEMANA'].dt.month
df_semanal_tipo['SEMANA_DEL_AÑO'] = df_semanal_tipo['FECHA_INICIO_SEMANA'].dt.isocalendar().week.astype('int64')
df_semanal_tipo['TRIMESTRE'] = df_semanal_tipo['FECHA_INICIO_SEMANA'].dt.quarter

df_semanal_tipo = df_semanal_tipo.sort_values(
    ['ZONA', 'TIPO_DELITO', 'FECHA_INICIO_SEMANA']
).reset_index(drop=True)

print(f"✅ Dataset creado: {len(df_semanal_tipo):,} registros")

# ============================================================
# PASO 2: FEATURES TEMPORALES - SIN FUGA
# ============================================================

print("\n[PASO 2] Generando features SIN FUGA...")

# Features cíclicas
df_semanal_tipo['MES_SIN'] = np.sin(2 * np.pi * df_semanal_tipo['MES'] / 12)
df_semanal_tipo['MES_COS'] = np.cos(2 * np.pi * df_semanal_tipo['MES'] / 12)
df_semanal_tipo['SEMANA_SIN'] = np.sin(2 * np.pi * df_semanal_tipo['SEMANA_DEL_AÑO'] / 52)
df_semanal_tipo['SEMANA_COS'] = np.cos(2 * np.pi * df_semanal_tipo['SEMANA_DEL_AÑO'] / 52)

# Lags (YA CORRECTOS)
for lag in [1, 2, 4, 8, 12]:
    df_semanal_tipo[f'DELITOS_LAG_{lag}'] = df_semanal_tipo.groupby(
        ['ZONA', 'TIPO_DELITO']
    )['TOTAL_DELITOS'].shift(lag)

# Rolling means (YA CORRECTOS)
for window in [4, 8, 12]:
    df_semanal_tipo[f'DELITOS_ROLLING_MEAN_{window}'] = df_semanal_tipo.groupby(
        ['ZONA', 'TIPO_DELITO']
    )['TOTAL_DELITOS'].transform(
        lambda x: x.shift(1).rolling(window, min_periods=1).mean()
    )

    df_semanal_tipo[f'DELITOS_ROLLING_STD_{window}'] = df_semanal_tipo.groupby(
        ['ZONA', 'TIPO_DELITO']
    )['TOTAL_DELITOS'].transform(
        lambda x: x.shift(1).rolling(window, min_periods=1).std()
    )

# ⚠️ CORRECCIÓN CRÍTICA: Promedios históricos CON shift(1)
print("   Corrigiendo PROMEDIO_HISTORICO_MES...")
df_semanal_tipo['PROMEDIO_HISTORICO_MES'] = df_semanal_tipo.groupby(
    ['ZONA', 'TIPO_DELITO', 'MES']
)['TOTAL_DELITOS'].transform(
    lambda x: x.shift(1).expanding().mean()  # ← shift(1) agregado
)

print("   Corrigiendo PROMEDIO_HISTORICO_TRIMESTRE...")
df_semanal_tipo['PROMEDIO_HISTORICO_TRIMESTRE'] = df_semanal_tipo.groupby(
    ['ZONA', 'TIPO_DELITO', 'TRIMESTRE']
)['TOTAL_DELITOS'].transform(
    lambda x: x.shift(1).expanding().mean()  # ← shift(1) agregado
)

# Tendencia (YA CORRECTO)
df_semanal_tipo['TENDENCIA'] = df_semanal_tipo.groupby(
    ['ZONA', 'TIPO_DELITO']
)['TOTAL_DELITOS'].pct_change(periods=4).fillna(0)

# Encoding
le_zona = LabelEncoder()
le_tipo = LabelEncoder()
df_semanal_tipo['ZONA_ENCODED'] = le_zona.fit_transform(df_semanal_tipo['ZONA'])
df_semanal_tipo['TIPO_DELITO_ENCODED'] = le_tipo.fit_transform(df_semanal_tipo['TIPO_DELITO'])

print(f"✅ Features generadas correctamente")

# ============================================================
# PASO 3: LIMPIAR Y DIVIDIR DATOS - CORREGIDO
# ============================================================

print("\n[PASO 3] Limpiando y dividiendo datos...")

df_limpio_tipo = df_semanal_tipo.dropna().copy()
print(f"Registros limpios: {len(df_limpio_tipo):,}")

# ⚠️ CORRECCIÓN CRÍTICA: Split temporal SIN overlap
df_limpio_tipo = df_limpio_tipo.sort_values('FECHA_INICIO_SEMANA').reset_index(drop=True)

# Obtener fechas únicas ordenadas
fechas_unicas = df_limpio_tipo['FECHA_INICIO_SEMANA'].unique()
fechas_unicas = np.sort(fechas_unicas)

# Split 80/20 por FECHAS (no por registros)
split_fecha_idx = int(len(fechas_unicas) * 0.8)
fecha_corte = fechas_unicas[split_fecha_idx]

train_tipo = df_limpio_tipo[df_limpio_tipo['FECHA_INICIO_SEMANA'] < fecha_corte].copy()
test_tipo = df_limpio_tipo[df_limpio_tipo['FECHA_INICIO_SEMANA'] >= fecha_corte].copy()

print(f"\nTrain: {len(train_tipo):,} registros")
print(f"  Período: {train_tipo['FECHA_INICIO_SEMANA'].min()} a {train_tipo['FECHA_INICIO_SEMANA'].max()}")

print(f"\nTest: {len(test_tipo):,} registros")
print(f"  Período: {test_tipo['FECHA_INICIO_SEMANA'].min()} a {test_tipo['FECHA_INICIO_SEMANA'].max()}")

# Verificar que NO hay overlap
assert train_tipo['FECHA_INICIO_SEMANA'].max() < test_tipo['FECHA_INICIO_SEMANA'].min(), "❌ HAY OVERLAP!"
print("✅ Separación temporal correcta (sin overlap)")

# ============================================================
# PASO 4: DEFINIR FEATURES Y ESTRATEGIA
# ============================================================

print("\n[PASO 4] Definiendo estrategia...")

features_tipo = [
    'ZONA_ENCODED',
    'TIPO_DELITO_ENCODED',
    'AÑO',
    'MES',
    'SEMANA_DEL_AÑO',
    'TRIMESTRE',
    'MES_SIN',
    'MES_COS',
    'SEMANA_SIN',
    'SEMANA_COS',
    'DELITOS_LAG_1',
    'DELITOS_LAG_2',
    'DELITOS_LAG_4',
    'DELITOS_LAG_8',
    'DELITOS_LAG_12',
    'DELITOS_ROLLING_MEAN_4',
    'DELITOS_ROLLING_STD_4',
    'DELITOS_ROLLING_MEAN_8',
    'DELITOS_ROLLING_STD_8',
    'DELITOS_ROLLING_MEAN_12',
    'DELITOS_ROLLING_STD_12',
    'PROMEDIO_HISTORICO_MES',
    'PROMEDIO_HISTORICO_TRIMESTRE',
    'TENDENCIA'
]

target = 'TOTAL_DELITOS'

# Estrategia
UMBRAL_INDIVIDUAL = 150
UMBRAL_TIPO = 50

combinaciones_train = train_tipo.groupby(['ZONA', 'TIPO_DELITO']).size().reset_index(name='N_REGISTROS')
combinaciones_train['ESTRATEGIA'] = combinaciones_train['N_REGISTROS'].apply(
    lambda x: 'INDIVIDUAL' if x >= UMBRAL_INDIVIDUAL
    else ('POR_TIPO' if x >= UMBRAL_TIPO else 'GLOBAL')
)

individuales = combinaciones_train[combinaciones_train['ESTRATEGIA'] == 'INDIVIDUAL']
print(f"\nCombinaciones INDIVIDUALES: {len(individuales)}")

# ============================================================
# PASO 5: ENTRENAR MODELOS
# ============================================================

print("\n" + "=" * 80)
print("[PASO 5] ENTRENANDO MODELOS (VERSIÓN SIN FUGA)")
print("=" * 80)

modelos_zona_tipo = {
    'INDIVIDUALES': {},
    'POR_TIPO': {},
    'GLOBAL': None
}

# 5A. Modelos individuales
print("\n[5A] Modelos INDIVIDUALES...")

for idx, row in individuales.iterrows():
    zona = row['ZONA']
    tipo = row['TIPO_DELITO']
    clave = f"{zona}_{tipo}"

    train_comb = train_tipo[
        (train_tipo['ZONA'] == zona) &
        (train_tipo['TIPO_DELITO'] == tipo)
    ].copy()

    test_comb = test_tipo[
        (test_tipo['ZONA'] == zona) &
        (test_tipo['TIPO_DELITO'] == tipo)
    ].copy()

    if len(test_comb) == 0:
        continue

    modelo = GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=7,
        min_samples_split=20,
        min_samples_leaf=10,
        subsample=0.8,
        random_state=42,
        verbose=0
    )

    modelo.fit(train_comb[features_tipo], train_comb[target])

    pred = modelo.predict(test_comb[features_tipo])
    mae = mean_absolute_error(test_comb[target], pred)
    r2 = r2_score(test_comb[target], pred)

    modelos_zona_tipo['INDIVIDUALES'][clave] = {
        'modelo': modelo,
        'zona': zona,
        'tipo': tipo,
        'mae': mae,
        'r2': r2,
        'n_train': len(train_comb),
        'n_test': len(test_comb)
    }

    print(f"{clave}: MAE={mae:.2f}, R²={r2:.4f}")

# 5B. Modelos por tipo
print("\n[5B] Modelos POR TIPO...")

for tipo in train_tipo['TIPO_DELITO'].unique():
    train_tipo_data = train_tipo[train_tipo['TIPO_DELITO'] == tipo].copy()
    test_tipo_data = test_tipo[test_tipo['TIPO_DELITO'] == tipo].copy()

    if len(test_tipo_data) == 0:
        continue

    modelo = GradientBoostingRegressor(
        n_estimators=400,
        learning_rate=0.03,
        max_depth=8,
        min_samples_split=30,
        min_samples_leaf=15,
        subsample=0.8,
        random_state=42,
        verbose=0
    )

    modelo.fit(train_tipo_data[features_tipo], train_tipo_data[target])

    pred = modelo.predict(test_tipo_data[features_tipo])
    mae = mean_absolute_error(test_tipo_data[target], pred)
    r2 = r2_score(test_tipo_data[target], pred)

    modelos_zona_tipo['POR_TIPO'][tipo] = {
        'modelo': modelo,
        'mae': mae,
        'r2': r2
    }

    print(f"{tipo}: MAE={mae:.2f}, R²={r2:.4f}")

# 5C. Modelo global
print("\n[5C] Modelo GLOBAL...")

modelo_global = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.02,
    max_depth=6,
    min_samples_split=50,
    min_samples_leaf=20,
    subsample=0.75,
    random_state=42,
    verbose=0
)

modelo_global.fit(train_tipo[features_tipo], train_tipo[target])

pred_global = modelo_global.predict(test_tipo[features_tipo])
mae_global = mean_absolute_error(test_tipo[target], pred_global)
r2_global = r2_score(test_tipo[target], pred_global)

modelos_zona_tipo['GLOBAL'] = {
    'modelo': modelo_global,
    'mae': mae_global,
    'r2': r2_global
}

print(f"GLOBAL: MAE={mae_global:.2f}, R²={r2_global:.4f}")

# Continúa en siguiente mensaje con predicciones y métricas...
# ============================================================
# PASO 6: GENERAR PREDICCIONES
# ============================================================

print("\n" + "=" * 80)
print("[PASO 6] GENERANDO PREDICCIONES (MODELO SIN FUGA)")
print("=" * 80)

predicciones_tipo = []

for idx, row in test_tipo.iterrows():
    zona = row['ZONA']
    tipo = row['TIPO_DELITO']
    clave = f"{zona}_{tipo}"

    # Seleccionar modelo jerárquicamente
    if clave in modelos_zona_tipo['INDIVIDUALES']:
        modelo_usar = modelos_zona_tipo['INDIVIDUALES'][clave]['modelo']
        tipo_modelo = 'INDIVIDUAL'
    elif tipo in modelos_zona_tipo['POR_TIPO']:
        modelo_usar = modelos_zona_tipo['POR_TIPO'][tipo]['modelo']
        tipo_modelo = 'POR_TIPO'
    else:
        modelo_usar = modelos_zona_tipo['GLOBAL']['modelo']
        tipo_modelo = 'GLOBAL'

    # Predecir
    X = row[features_tipo].values.reshape(1, -1)
    pred = modelo_usar.predict(X)[0]

    predicciones_tipo.append({
        'ZONA': zona,
        'TIPO_DELITO': tipo,
        'FECHA_INICIO': row['FECHA_INICIO_SEMANA'],
        'AÑO': row['AÑO'],
        'MES': row['MES'],
        'SEMANA': row['SEMANA_DEL_AÑO'],
        'TOTAL_DELITOS_REAL': row['TOTAL_DELITOS'],
        'TOTAL_DELITOS_PREDICHO': round(pred, 2),
        'ERROR_ABSOLUTO': round(abs(row['TOTAL_DELITOS'] - pred), 2),
        'ERROR_PORCENTUAL': round(abs(row['TOTAL_DELITOS'] - pred) / max(row['TOTAL_DELITOS'], 1) * 100, 2),
        'TIPO_MODELO': tipo_modelo
    })

df_predicciones_tipo = pd.DataFrame(predicciones_tipo)

print(f"✅ Predicciones generadas: {len(df_predicciones_tipo):,}")

# ============================================================
# PASO 7: MÉTRICAS FINALES
# ============================================================

print("\n" + "=" * 80)
print("[PASO 7] MÉTRICAS FINALES (MODELO SIN FUGA)")
print("=" * 80)

# Métricas globales
mae_final_tipo = df_predicciones_tipo['ERROR_ABSOLUTO'].mean()
rmse_final_tipo = np.sqrt(mean_squared_error(
    df_predicciones_tipo['TOTAL_DELITOS_REAL'],
    df_predicciones_tipo['TOTAL_DELITOS_PREDICHO']
))
r2_final_tipo = r2_score(
    df_predicciones_tipo['TOTAL_DELITOS_REAL'],
    df_predicciones_tipo['TOTAL_DELITOS_PREDICHO']
)

print(f"\n📊 MÉTRICAS GLOBALES (SIN FUGA):")
print(f"   MAE:  {mae_final_tipo:.2f} delitos")
print(f"   RMSE: {rmse_final_tipo:.2f}")
print(f"   R²:   {r2_final_tipo:.4f}")

# Comparación con modelo con fuga
print(f"\n📉 COMPARACIÓN CON MODELO CON FUGA:")
print(f"   MAE:  0.61 → {mae_final_tipo:.2f} (cambio: {((mae_final_tipo/0.61 - 1)*100):+.1f}%)")
print(f"   R²:   0.9908 → {r2_final_tipo:.4f} (cambio: {((r2_final_tipo/0.9908 - 1)*100):+.1f}%)")

# Métricas por ZONA + TIPO_DELITO
print(f"\n📍 MÉTRICAS POR ZONA + TIPO DE DELITO:")
print("─" * 110)

metricas_zona_tipo = []

for (zona, tipo), grupo in df_predicciones_tipo.groupby(['ZONA', 'TIPO_DELITO']):
    mae_comb = grupo['ERROR_ABSOLUTO'].mean()
    rmse_comb = np.sqrt(mean_squared_error(grupo['TOTAL_DELITOS_REAL'], grupo['TOTAL_DELITOS_PREDICHO']))
    r2_comb = r2_score(grupo['TOTAL_DELITOS_REAL'], grupo['TOTAL_DELITOS_PREDICHO'])
    promedio_real = grupo['TOTAL_DELITOS_REAL'].mean()
    promedio_pred = grupo['TOTAL_DELITOS_PREDICHO'].mean()
    error_pct = grupo['ERROR_PORCENTUAL'].mean()
    tipo_modelo = grupo['TIPO_MODELO'].mode()[0] if len(grupo) > 0 else 'N/A'

    metricas_zona_tipo.append({
        'ZONA': zona,
        'TIPO_DELITO': tipo,
        'TIPO_MODELO': tipo_modelo,
        'PROMEDIO_REAL': round(promedio_real, 2),
        'PROMEDIO_PREDICHO': round(promedio_pred, 2),
        'MAE': round(mae_comb, 2),
        'RMSE': round(rmse_comb, 2),
        'R2': round(r2_comb, 4),
        'ERROR_%': round(error_pct, 1),
        'N_REGISTROS': len(grupo)
    })

df_metricas_zona_tipo = pd.DataFrame(metricas_zona_tipo)
df_metricas_zona_tipo = df_metricas_zona_tipo.sort_values(['ZONA', 'PROMEDIO_REAL'], ascending=[True, False])

print(df_metricas_zona_tipo.to_string(index=False))

# Métricas por TIPO DE DELITO
print(f"\n📊 RESUMEN POR TIPO DE DELITO:")
print("─" * 90)

resumen_tipo = df_predicciones_tipo.groupby('TIPO_DELITO').apply(lambda x: pd.Series({
    'N_REGISTROS': len(x),
    'PROMEDIO_REAL': x['TOTAL_DELITOS_REAL'].mean(),
    'PROMEDIO_PREDICHO': x['TOTAL_DELITOS_PREDICHO'].mean(),
    'MAE': x['ERROR_ABSOLUTO'].mean(),
    'R2': r2_score(x['TOTAL_DELITOS_REAL'], x['TOTAL_DELITOS_PREDICHO']),
    'ERROR_%': x['ERROR_PORCENTUAL'].mean()
})).round(2)

print(resumen_tipo)

# Métricas por ZONA
print(f"\n📍 RESUMEN POR ZONA:")
print("─" * 90)

resumen_zona = df_predicciones_tipo.groupby('ZONA').apply(lambda x: pd.Series({
    'N_REGISTROS': len(x),
    'PROMEDIO_REAL': x['TOTAL_DELITOS_REAL'].mean(),
    'PROMEDIO_PREDICHO': x['TOTAL_DELITOS_PREDICHO'].mean(),
    'MAE': x['ERROR_ABSOLUTO'].mean(),
    'R2': r2_score(x['TOTAL_DELITOS_REAL'], x['TOTAL_DELITOS_PREDICHO']),
    'ERROR_%': x['ERROR_PORCENTUAL'].mean()
})).round(2)

resumen_zona = resumen_zona.sort_values('PROMEDIO_REAL', ascending=False)
print(resumen_zona)

# ============================================================
# PASO 8: ANÁLISIS DE VIABILIDAD ACTUALIZADO
# ============================================================

print("\n" + "=" * 80)
print("[PASO 8] ANÁLISIS DE VIABILIDAD (MODELO CORREGIDO)")
print("=" * 80)

def clasificar_viabilidad_v2(row):
    """Clasificación de viabilidad más estricta"""
    mae = row['MAE']
    r2 = row['R2']
    promedio = row['PROMEDIO_REAL']
    error_pct = row['ERROR_%']

    # Criterios más realistas
    if r2 >= 0.5 and error_pct < 30:
        return '🟢 ALTA'
    elif r2 >= 0.3 and error_pct < 50:
        return '🟡 MEDIA'
    elif mae < 2 and promedio < 5:
        return '🟡 MEDIA'  # Zonas pequeñas con MAE bajo
    else:
        return '🔴 BAJA'

df_metricas_zona_tipo['VIABILIDAD'] = df_metricas_zona_tipo.apply(clasificar_viabilidad_v2, axis=1)

print("\n📋 CLASIFICACIÓN DE VIABILIDAD (ACTUALIZADA):")
print("─" * 110)
print(df_metricas_zona_tipo[['ZONA', 'TIPO_DELITO', 'MAE', 'R2', 'ERROR_%', 'VIABILIDAD']].to_string(index=False))

print("\n📊 Distribución de viabilidad:")
print(df_metricas_zona_tipo['VIABILIDAD'].value_counts().sort_index())

# Mejores y peores
print("\n🏆 TOP 5 MEJORES COMBINACIONES (por R²):")
top5 = df_metricas_zona_tipo.nlargest(5, 'R2')[['ZONA', 'TIPO_DELITO', 'R2', 'MAE', 'VIABILIDAD']]
print(top5.to_string(index=False))

print("\n⚠️ TOP 5 PEORES COMBINACIONES (por R²):")
bottom5 = df_metricas_zona_tipo.nsmallest(5, 'R2')[['ZONA', 'TIPO_DELITO', 'R2', 'MAE', 'ERROR_%', 'VIABILIDAD']]
print(bottom5.to_string(index=False))

# ============================================================
# PASO 9: ANÁLISIS DE ERRORES
# ============================================================

print("\n" + "=" * 80)
print("[PASO 9] ANÁLISIS DE ERRORES")
print("=" * 80)

# Top 10 mayores errores
print("\n🔴 TOP 10 MAYORES ERRORES:")
top_errores = df_predicciones_tipo.nlargest(10, 'ERROR_ABSOLUTO')[
    ['ZONA', 'TIPO_DELITO', 'FECHA_INICIO', 'TOTAL_DELITOS_REAL', 'TOTAL_DELITOS_PREDICHO', 'ERROR_ABSOLUTO']
]
print(top_errores.to_string(index=False))

# Distribución de errores
print("\n📊 ESTADÍSTICAS DE ERROR ABSOLUTO:")
print(f"   Mínimo: {df_predicciones_tipo['ERROR_ABSOLUTO'].min():.2f}")
print(f"   Q25:    {df_predicciones_tipo['ERROR_ABSOLUTO'].quantile(0.25):.2f}")
print(f"   Mediana: {df_predicciones_tipo['ERROR_ABSOLUTO'].median():.2f}")
print(f"   Q75:    {df_predicciones_tipo['ERROR_ABSOLUTO'].quantile(0.75):.2f}")
print(f"   Máximo: {df_predicciones_tipo['ERROR_ABSOLUTO'].max():.2f}")

# ============================================================
# PASO 10: RECOMENDACIONES ACTUALIZADAS
# ============================================================

print("\n" + "=" * 80)
print("[PASO 10] RECOMENDACIONES PARA USO PÚBLICO (ACTUALIZADAS)")
print("=" * 80)

recomendaciones = []

for idx, row in df_metricas_zona_tipo.iterrows():
    zona = row['ZONA']
    tipo = row['TIPO_DELITO']
    viabilidad = row['VIABILIDAD']
    mae = row['MAE']
    promedio = row['PROMEDIO_REAL']
    r2 = row['R2']

    # Generar recomendación actualizada
    if '🟢' in viabilidad:
        mensaje_tipo = 'USAR RANGOS AMPLIOS'
        rango_min = max(0, promedio - mae * 2)
        rango_max = promedio + mae * 2
        ejemplo = f"Se esperan entre {int(rango_min)}-{int(rango_max)} casos de {tipo.lower().replace('_', ' ')} (±{int(mae*2)} casos)"
    elif '🟡' in viabilidad:
        mensaje_tipo = 'USAR TENDENCIAS Y PROMEDIOS'
        if promedio < 3:
            ejemplo = f"Niveles bajos de {tipo.lower().replace('_', ' ')} (promedio histórico: {int(promedio)} casos/semana)"
        else:
            ejemplo = f"Promedio histórico de {int(promedio)}±{int(mae)} casos de {tipo.lower().replace('_', ' ')}"
    else:
        mensaje_tipo = 'SOLO DATOS HISTÓRICOS'
        ejemplo = f"Reportar únicamente datos históricos de {tipo.lower().replace('_', ' ')} (predicción no confiable)"

    recomendaciones.append({
        'ZONA': zona,
        'TIPO_DELITO': tipo,
        'VIABILIDAD': viabilidad,
        'R2': r2,
        'MAE': mae,
        'RECOMENDACION': mensaje_tipo,
        'EJEMPLO_COMUNICACION': ejemplo
    })

df_recomendaciones = pd.DataFrame(recomendaciones)

print("\n📢 GUÍA DE COMUNICACIÓN ACTUALIZADA:")
print("─" * 120)
print(df_recomendaciones[['ZONA', 'TIPO_DELITO', 'VIABILIDAD', 'R2', 'RECOMENDACION']].to_string(index=False))

# Ejemplos específicos
print("\n💬 EJEMPLOS DE MENSAJES PARA LA CIUDADANÍA (VERSIÓN CORREGIDA):")
print("─" * 120)

# Agrupar por viabilidad
for viab in ['🟢 ALTA', '🟡 MEDIA', '🔴 BAJA']:
    ejemplos_viab = df_recomendaciones[df_recomendaciones['VIABILIDAD'] == viab].head(3)
    if len(ejemplos_viab) > 0:
        print(f"\n{viab}:")
        for idx, row in ejemplos_viab.iterrows():
            print(f"  • {row['ZONA']} - {row['TIPO_DELITO']}:")
            print(f"    {row['EJEMPLO_COMUNICACION']}")

# ============================================================
# PASO 11: EXPORTAR RESULTADOS
# ============================================================

print("\n" + "=" * 80)
print("[PASO 11] EXPORTANDO RESULTADOS (VERSIÓN SIN FUGA)")
print("=" * 80)

# 1. Predicciones
archivo_pred = '001 predicciones_zona_tipo_SIN_FUGA.csv'
df_predicciones_tipo.to_csv(archivo_pred, index=False, encoding='utf-8-sig')
print(f"\n✅ {archivo_pred} ({len(df_predicciones_tipo):,} registros)")

# 2. Métricas
archivo_metricas = '001 metricas_zona_tipo_SIN_FUGA.csv'
df_metricas_zona_tipo.to_csv(archivo_metricas, index=False, encoding='utf-8-sig')
print(f"✅ {archivo_metricas} ({len(df_metricas_zona_tipo)} combinaciones)")

# 3. Recomendaciones
archivo_recom = '001 recomendaciones_SIN_FUGA.csv'
df_recomendaciones.to_csv(archivo_recom, index=False, encoding='utf-8-sig')
print(f"✅ {archivo_recom}")

# 4. Resumen por tipo
archivo_tipo = '001 resumen_tipo_SIN_FUGA.csv'
resumen_tipo.to_csv(archivo_tipo, encoding='utf-8-sig')
print(f"✅ {archivo_tipo}")

# 5. Resumen por zona
archivo_zona = '001 resumen_zona_SIN_FUGA.csv'
resumen_zona.to_csv(archivo_zona, encoding='utf-8-sig')
print(f"✅ {archivo_zona}")

archivo_general = '001 General.csv'
df_con_zona.to_csv(archivo_general, encoding='utf-8-sig')
print(f"✅ {archivo_general}")


# 7. Resumen ejecutivo
resumen_ejecutivo = pd.DataFrame([{
    'FECHA_GENERACION': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'VERSION': 'SIN FUGA - v2.0',
    'TOTAL_PREDICCIONES': len(df_predicciones_tipo),
    'COMBINACIONES_ANALIZADAS': len(df_metricas_zona_tipo),
    'MAE_GLOBAL': round(mae_final_tipo, 2),
    'RMSE_GLOBAL': round(rmse_final_tipo, 2),
    'R2_GLOBAL': round(r2_final_tipo, 4),
    'MODELOS_INDIVIDUALES': len(modelos_zona_tipo['INDIVIDUALES']),
    'MODELOS_POR_TIPO': len(modelos_zona_tipo['POR_TIPO']),
    'VIABILIDAD_ALTA': (df_metricas_zona_tipo['VIABILIDAD'] == '🟢 ALTA').sum(),
    'VIABILIDAD_MEDIA': (df_metricas_zona_tipo['VIABILIDAD'] == '🟡 MEDIA').sum(),
    'VIABILIDAD_BAJA': (df_metricas_zona_tipo['VIABILIDAD'] == '🔴 BAJA').sum(),
    'CORRECCION_1': 'PROMEDIO_HISTORICO con shift(1)',
    'CORRECCION_2': 'Split temporal sin overlap'
}])

archivo_ejecutivo = '001 resumen_ejecutivo_SIN_FUGA.csv'
resumen_ejecutivo.to_csv(archivo_ejecutivo, index=False, encoding='utf-8-sig')
print(f"✅ {archivo_ejecutivo}")


# ============================================================
# RESUMEN FINAL
# ============================================================

print("\n" + "=" * 80)
print("✅ MODELO SIN FUGA COMPLETADO")
print("=" * 80)


for i, archivo in enumerate(archivos, 1):
    if os.path.exists(archivo):
        tamaño = os.path.getsize(archivo) / 1024
        print(f"   {i}. {archivo} ({tamaño:.1f} KB)")

print("\n📊 MÉTRICAS FINALES (SIN FUGA):")
print(f"   MAE:  {mae_final_tipo:.2f} delitos")
print(f"   RMSE: {rmse_final_tipo:.2f}")
print(f"   R²:   {r2_final_tipo:.4f}")

p
print("\n🎯 VIABILIDAD FINAL:")
print(f"   Alta (🟢): {(df_metricas_zona_tipo['VIABILIDAD'] == '🟢 ALTA').sum()} combinaciones")
print(f"   Media (🟡): {(df_metricas_zona_tipo['VIABILIDAD'] == '🟡 MEDIA').sum()} combinaciones")
print(f"   Baja (🔴): {(df_metricas_zona_tipo['VIABILIDAD'] == '🔴 BAJA').sum()} combinaciones")



MODELO ZONA + TIPO DELITO - VERSIÓN CORREGIDA (SIN FUGA)

[PASO 1] Creando dataset agregado...
✅ Dataset creado: 9,773 registros

[PASO 2] Generando features SIN FUGA...
   Corrigiendo PROMEDIO_HISTORICO_MES...
   Corrigiendo PROMEDIO_HISTORICO_TRIMESTRE...
✅ Features generadas correctamente

[PASO 3] Limpiando y dividiendo datos...
Registros limpios: 9,393

Train: 7,516 registros
  Período: 2010-03-22 00:00:00 a 2022-05-23 00:00:00

Test: 1,877 registros
  Período: 2022-05-30 00:00:00 a 2025-05-26 00:00:00
✅ Separación temporal correcta (sin overlap)

[PASO 4] Definiendo estrategia...

Combinaciones INDIVIDUALES: 15

[PASO 5] ENTRENANDO MODELOS (VERSIÓN SIN FUGA)

[5A] Modelos INDIVIDUALES...
Comunera_DELITO_SEXUAL: MAE=0.09, R²=0.9354
Comunera_VIOLENCIA_INTRAFAMILIAR: MAE=0.40, R²=0.8971
García-Rovira_DELITO_SEXUAL: MAE=0.07, R²=0.9528
García-Rovira_VIOLENCIA_INTRAFAMILIAR: MAE=0.19, R²=0.9011
Guanentá_DELITO_SEXUAL: MAE=0.07, R²=0.9618
Guanentá_VIOLENCIA_INTRAFAMILIAR: MAE=0.26, R²=

NameError: name 'archivos' is not defined

###Modelado bucaramanga

In [ ]:
import pandas as pd
import numpy as np

print("="*60)
print("FASE 1: PREPARACIÓN DE DATOS")
print("="*60)

# 1. CREAR COLUMNA HORA NUMÉRICA (CORREGIDO)
print("\n[1/4] Creando columna HORA numérica...")

# Primero verificamos qué formato tiene HORA HECHO
print(f"Tipo de dato actual: {df_bucaramanga_clean['HORA HECHO'].dtype}")
print(f"Ejemplo de valores:")
print(df_bucaramanga_clean['HORA HECHO'].head())

# Convertir a datetime manejando diferentes formatos
df_bucaramanga_clean['HORA'] = pd.to_datetime(
    df_bucaramanga_clean['HORA HECHO'],
    format='mixed',
    errors='coerce'
).dt.hour

print(f"✓ Columna HORA creada (rango: {df_bucaramanga_clean['HORA'].min()} - {df_bucaramanga_clean['HORA'].max()})")

# Verificar si hubo errores en la conversión
nulos_hora = df_bucaramanga_clean['HORA'].isnull().sum()
if nulos_hora > 0:
    print(f"⚠️ ADVERTENCIA: {nulos_hora} valores no se pudieron convertir")
    print("Valores problemáticos:")
    print(df_bucaramanga_clean[df_bucaramanga_clean['HORA'].isnull()]['HORA HECHO'].unique()[:10])

# 2. CREAR COLUMNA BLOQUE_HORARIO
print("\n[2/4] Creando columna BLOQUE_HORARIO...")
def asignar_bloque(hora):
    if pd.isna(hora):
        return 'DESCONOCIDO'
    elif 0 <= hora < 6:
        return 'MADRUGADA'
    elif 6 <= hora < 12:
        return 'MAÑANA'
    elif 12 <= hora < 18:
        return 'TARDE'
    else:  # 18 <= hora < 24
        return 'NOCHE'

df_bucaramanga_clean['BLOQUE_HORARIO'] = df_bucaramanga_clean['HORA'].apply(asignar_bloque)
print("✓ Columna BLOQUE_HORARIO creada")

# 3. VERIFICAR CATEGORÍAS DE DELITO
print("\n[3/4] Verificando CATEGORIA DELITO...")
print("\n--- Distribución de Categorías ---")
print(df_bucaramanga_clean['CATEGORIA DELITO'].value_counts())
print(f"\nTotal categorías únicas: {df_bucaramanga_clean['CATEGORIA DELITO'].nunique()}")

# 4. VERIFICAR NOMBRES DE COMUNAS
print("\n[4/4] Verificando NOMBRE COMUNA...")
print("\n--- Comunas Únicas (ordenadas) ---")
comunas_unicas = sorted(df_bucaramanga_clean['NOMBRE COMUNA'].unique())
for i, comuna in enumerate(comunas_unicas, 1):
    print(f"{i:2}. '{comuna}'")
print(f"\nTotal comunas: {df_bucaramanga_clean['NOMBRE COMUNA'].nunique()}")

# RESUMEN GENERAL
print("\n" + "="*60)
print("RESUMEN DE DATOS PREPARADOS")
print("="*60)
print(f"Total registros: {len(df_bucaramanga_clean):,}")
print(f"Período: {df_bucaramanga_clean['FECHA HECHO'].min().date()} a {df_bucaramanga_clean['FECHA HECHO'].max().date()}")
print(f"\nDistribución por BLOQUE_HORARIO:")
print(df_bucaramanga_clean['BLOQUE_HORARIO'].value_counts())
print(f"\nDistribución por día de semana:")
print(df_bucaramanga_clean['DIA NOMBRE'].value_counts())

# VERIFICACIÓN DE CALIDAD
print("\n" + "="*60)
print("VERIFICACIÓN DE CALIDAD")
print("="*60)
print(f"Valores nulos en HORA: {df_bucaramanga_clean['HORA'].isnull().sum()}")
print(f"Valores nulos en BLOQUE_HORARIO: {df_bucaramanga_clean['BLOQUE_HORARIO'].isnull().sum()}")
print(f"Valores en DESCONOCIDO: {(df_bucaramanga_clean['BLOQUE_HORARIO'] == 'DESCONOCIDO').sum()}")
print(f"Valores nulos en CATEGORIA DELITO: {df_bucaramanga_clean['CATEGORIA DELITO'].isnull().sum()}")
print(f"Valores nulos en NOMBRE COMUNA: {df_bucaramanga_clean['NOMBRE COMUNA'].isnull().sum()}")

# MUESTRA DE DATOS TRANSFORMADOS
print("\n" + "="*60)
print("MUESTRA DE DATOS TRANSFORMADOS (primeras 10 filas)")
print("="*60)
print(df_bucaramanga_clean[['FECHA HECHO', 'HORA HECHO', 'HORA', 'BLOQUE_HORARIO',
          'CATEGORIA DELITO', 'NOMBRE COMUNA', 'DIA NOMBRE']].head(10))

# Distribución detallada por HORA
print("\n" + "="*60)
print("DISTRIBUCIÓN DE DELITOS POR HORA (0-23)")
print("="*60)
print(df_bucaramanga_clean['HORA'].value_counts().sort_index())


In [ ]:
import pandas as pd
import numpy as np

print("="*60)
print("FASE 2: INGENIERÍA DE FEATURES Y AGREGACIÓN")
print("="*60)

# 1. CREAR FEATURES ADICIONALES
print("\n[1/5] Creando features adicionales...")

# Día de la semana como número (0=lunes, 6=domingo)
df_bucaramanga_clean['DIA_SEMANA_NUM'] = pd.to_datetime(df_bucaramanga_clean['FECHA HECHO']).dt.dayofweek

# Año y mes para análisis temporal
df_bucaramanga_clean['AÑO_NUM'] = pd.to_datetime(df_bucaramanga_clean['FECHA HECHO']).dt.year
df_bucaramanga_clean['MES_NUM'] = pd.to_datetime(df_bucaramanga_clean['FECHA HECHO']).dt.month

# Indicador de fin de semana
df_bucaramanga_clean['ES_FIN_SEMANA'] = df_bucaramanga_clean['DIA_SEMANA_NUM'].isin([5, 6]).astype(int)

print("✓ Features creadas: DIA_SEMANA_NUM, AÑO_NUM, MES_NUM, ES_FIN_SEMANA")

# 2. AGREGACIÓN PRINCIPAL: Contar delitos por combinación
print("\n[2/5] Agregando datos por COMUNA + CATEGORIA + DIA + BLOQUE...")

df_agregado = df_bucaramanga_clean.groupby([
    'NOMBRE COMUNA',
    'CATEGORIA DELITO',
    'DIA NOMBRE',
    'DIA_SEMANA_NUM',
    'BLOQUE_HORARIO'
]).agg({
    'CANTIDAD': 'count'  # Contamos delitos
}).reset_index()

# Renombrar columna para claridad
df_agregado.rename(columns={'CANTIDAD': 'TOTAL_DELITOS'}, inplace=True)

print(f"✓ Tabla agregada creada: {len(df_agregado):,} combinaciones únicas")
print(f"  Columnas: {list(df_agregado.columns)}")

# 3. CREAR FEATURES HISTÓRICAS (Promedios globales)
print("\n[3/5] Calculando estadísticas históricas...")

# Promedio de delitos por comuna
promedio_por_comuna = df_bucaramanga_clean.groupby('NOMBRE COMUNA').size().reset_index(name='PROMEDIO_COMUNA')
df_agregado = df_agregado.merge(promedio_por_comuna, on='NOMBRE COMUNA', how='left')

# Promedio de delitos por bloque horario
promedio_por_bloque = df_bucaramanga_clean.groupby('BLOQUE_HORARIO').size().reset_index(name='PROMEDIO_BLOQUE')
df_agregado = df_agregado.merge(promedio_por_bloque, on='BLOQUE_HORARIO', how='left')

# Promedio de delitos por categoría
promedio_por_categoria = df_bucaramanga_clean.groupby('CATEGORIA DELITO').size().reset_index(name='PROMEDIO_CATEGORIA')
df_agregado = df_agregado.merge(promedio_por_categoria, on='CATEGORIA DELITO', how='left')

# Promedio de delitos por día de semana
promedio_por_dia = df_bucaramanga_clean.groupby('DIA NOMBRE').size().reset_index(name='PROMEDIO_DIA')
df_agregado = df_agregado.merge(promedio_por_dia, on='DIA NOMBRE', how='left')

print("✓ Features históricas agregadas:")
print("  - PROMEDIO_COMUNA")
print("  - PROMEDIO_BLOQUE")
print("  - PROMEDIO_CATEGORIA")
print("  - PROMEDIO_DIA")

# 4. NORMALIZAR TOTAL_DELITOS (para calcular probabilidades)
print("\n[4/5] Calculando probabilidades relativas...")

# Calcular probabilidad por comuna (qué tan riesgosa es cada comuna)
total_delitos_global = df_bucaramanga_clean.shape[0]
df_agregado['PROBABILIDAD_RELATIVA'] = df_agregado['TOTAL_DELITOS'] / total_delitos_global

# Calcular percentiles para definir umbrales
percentil_33 = df_agregado['TOTAL_DELITOS'].quantile(0.33)
percentil_66 = df_agregado['TOTAL_DELITOS'].quantile(0.66)

print(f"✓ Percentiles calculados:")
print(f"  - 33%: {percentil_33:.0f} delitos")
print(f"  - 66%: {percentil_66:.0f} delitos")

# 5. ASIGNAR CATEGORÍA DE RIESGO INICIAL (será mejorada con el modelo)
print("\n[5/5] Asignando categoría de riesgo preliminar...")

def asignar_riesgo(total_delitos):
    if total_delitos <= percentil_33:
        return 'BAJO'
    elif total_delitos <= percentil_66:
        return 'MEDIO'
    else:
        return 'ALTO'

df_agregado['CATEGORIA_RIESGO_PRELIMINAR'] = df_agregado['TOTAL_DELITOS'].apply(asignar_riesgo)

print("✓ Categorías asignadas:")
print(df_agregado['CATEGORIA_RIESGO_PRELIMINAR'].value_counts())

# RESUMEN FINAL
print("\n" + "="*60)
print("RESUMEN DE TABLA AGREGADA")
print("="*60)
print(f"Total filas: {len(df_agregado):,}")
print(f"Total columnas: {len(df_agregado.columns)}")
print(f"\nColumnas finales:")
for col in df_agregado.columns:
    print(f"  - {col}")

print(f"\nEstadísticas de TOTAL_DELITOS:")
print(df_agregado['TOTAL_DELITOS'].describe())

# MUESTRA DE DATOS
print("\n" + "="*60)
print("MUESTRA DE TABLA AGREGADA (Top 10 combinaciones)")
print("="*60)
print(df_agregado.nlargest(10, 'TOTAL_DELITOS')[
    ['NOMBRE COMUNA', 'CATEGORIA DELITO', 'DIA NOMBRE', 'BLOQUE_HORARIO',
     'TOTAL_DELITOS', 'CATEGORIA_RIESGO_PRELIMINAR']
])

# Verificar que no haya combinaciones faltantes importantes
print("\n" + "="*60)
print("VERIFICACIÓN DE COBERTURA")
print("="*60)
print(f"Comunas únicas en agregado: {df_agregado['NOMBRE COMUNA'].nunique()}/21")
print(f"Categorías únicas: {df_agregado['CATEGORIA DELITO'].nunique()}/8")
print(f"Días únicos: {df_agregado['DIA NOMBRE'].nunique()}/7")
print(f"Bloques únicos: {df_agregado['BLOQUE_HORARIO'].nunique()}/4")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

print("="*60)
print("FASE 3: MODELADO DE MACHINE LEARNING")
print("="*60)

# 1. PREPARAR DATOS PARA EL MODELO
print("\n[1/6] Preparando datos para el modelo...")

# Crear copia para no modificar el original
df_modelo = df_agregado.copy()

# Codificar variables categóricas
label_encoders = {}
columnas_categoricas = ['NOMBRE COMUNA', 'CATEGORIA DELITO', 'DIA NOMBRE', 'BLOQUE_HORARIO']

for col in columnas_categoricas:
    le = LabelEncoder()
    df_modelo[f'{col}_ENCODED'] = le.fit_transform(df_modelo[col])
    label_encoders[col] = le
    print(f"  ✓ {col}: {df_modelo[col].nunique()} categorías codificadas")

# Codificar variable objetivo
le_target = LabelEncoder()
df_modelo['TARGET'] = le_target.fit_transform(df_modelo['CATEGORIA_RIESGO_PRELIMINAR'])
print(f"\n  ✓ TARGET codificado: {dict(zip(le_target.classes_, le_target.transform(le_target.classes_)))}")

# 2. DEFINIR FEATURES Y TARGET
print("\n[2/6] Definiendo features y target...")

# Features para el modelo
feature_columns = [
    'NOMBRE COMUNA_ENCODED',
    'CATEGORIA DELITO_ENCODED',
    'DIA NOMBRE_ENCODED',
    'DIA_SEMANA_NUM',
    'BLOQUE_HORARIO_ENCODED',
    'PROMEDIO_COMUNA',
    'PROMEDIO_BLOQUE',
    'PROMEDIO_CATEGORIA',
    'PROMEDIO_DIA'
]

X = df_modelo[feature_columns]
y = df_modelo['TARGET']

print(f"  ✓ Features: {len(feature_columns)} columnas")
print(f"  ✓ Registros totales: {len(X):,}")
print(f"  ✓ Distribución del target:")
for clase, nombre in zip(le_target.transform(le_target.classes_), le_target.classes_):
    print(f"    - {nombre}: {(y == clase).sum():,} ({(y == clase).sum()/len(y)*100:.1f}%)")

# 3. DIVIDIR EN TRAIN Y TEST
print("\n[3/6] Dividiendo datos en Train/Test...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Mantener proporción de clases
)

print(f"  ✓ Train: {len(X_train):,} registros ({len(X_train)/len(X)*100:.1f}%)")
print(f"  ✓ Test:  {len(X_test):,} registros ({len(X_test)/len(X)*100:.1f}%)")

# 4. ENTRENAR MODELO RANDOM FOREST
print("\n[4/6] Entrenando modelo Random Forest...")

modelo_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # Manejar desbalance de clases
)

modelo_rf.fit(X_train, y_train)
print("  ✓ Modelo entrenado exitosamente")

# 5. EVALUAR MODELO
print("\n[5/6] Evaluando modelo...")

# Predicciones
y_pred_train = modelo_rf.predict(X_train)
y_pred_test = modelo_rf.predict(X_test)

# Accuracy
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

print(f"\n  ✓ Accuracy en Train: {acc_train:.4f} ({acc_train*100:.2f}%)")
print(f"  ✓ Accuracy en Test:  {acc_test:.4f} ({acc_test*100:.2f}%)")

# Reporte de clasificación
print("\n" + "="*60)
print("REPORTE DE CLASIFICACIÓN (Test Set)")
print("="*60)
print(classification_report(
    y_test,
    y_pred_test,
    target_names=le_target.classes_,
    digits=3
))

# Matriz de confusión
print("\n" + "="*60)
print("MATRIZ DE CONFUSIÓN (Test Set)")
print("="*60)
cm = confusion_matrix(y_test, y_pred_test)
cm_df = pd.DataFrame(
    cm,
    index=le_target.classes_,
    columns=[f'Pred_{c}' for c in le_target.classes_]
)
print(cm_df)

# 6. IMPORTANCIA DE FEATURES
print("\n[6/6] Analizando importancia de features...")

feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': modelo_rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n  Feature Importance:")
for idx, row in feature_importance.iterrows():
    print(f"    {row['Feature']:30s}: {row['Importance']:.4f}")

# RESUMEN FINAL
print("\n" + "="*60)
print("RESUMEN DEL MODELO")
print("="*60)
print(f"Algoritmo: Random Forest")
print(f"Features: {len(feature_columns)}")
print(f"Clases: {len(le_target.classes_)} (ALTO, BAJO, MEDIO)")
print(f"Accuracy Test: {acc_test*100:.2f}%")
print(f"Registros entrenamiento: {len(X_train):,}")
print(f"Registros prueba: {len(X_test):,}")

# Verificar si hay overfitting
diff_acc = acc_train - acc_test
if diff_acc > 0.10:
    print(f"\n⚠️ ADVERTENCIA: Posible overfitting detectado")
    print(f"   Diferencia Train-Test: {diff_acc*100:.2f}%")
else:
    print(f"\n✅ Modelo balanceado (diff: {diff_acc*100:.2f}%)")



In [ ]:
print("="*60)
print("CORRECCIÓN FINAL: QUITAR 'COMUNA' DUPLICADO")
print("="*60)

# Función para limpiar el "Comuna Comuna" duplicado
def limpiar_comuna_duplicada(nombre):
    # Si tiene "Comuna Comuna", quitar el primero
    if nombre.startswith('Comuna Comuna '):
        return nombre.replace('Comuna Comuna ', 'Comuna ', 1)
    return nombre

# Aplicar limpieza
print("\n[LIMPIANDO] Quitando 'Comuna' duplicado...")

df_bucaramanga_clean['NOMBRE COMUNA'] = df_bucaramanga_clean['NOMBRE COMUNA'].apply(limpiar_comuna_duplicada)
print("✓ df_bucaramanga_clean limpio")

if 'df_agregado' in globals():
    df_agregado['NOMBRE COMUNA'] = df_agregado['NOMBRE COMUNA'].apply(limpiar_comuna_duplicada)
    print("✓ df_agregado limpio")

if 'df_completo' in globals():
    df_completo['NOMBRE COMUNA'] = df_completo['NOMBRE COMUNA'].apply(limpiar_comuna_duplicada)
    print("✓ df_completo limpio")

if 'df_powerbi' in globals():
    df_powerbi['COMUNA'] = df_powerbi['COMUNA'].apply(limpiar_comuna_duplicada)
    print("✓ df_powerbi limpio")

# Correcciones manuales adicionales
print("\n[CORRIGIENDO] Ajustes finales...")

# Corrección manual de nombres específicos
correcciones_manuales = {
    'Comuna Cabecera Del Llano': 'Comuna Cabecera del Llano',  # "del" minúscula
    'Comuna Ciudadela Rela De Minas': 'Comuna Ciudadela Real de Minas',  # "Real de" correcto
    'Comuna No Disponible': 'SIN GEOCODIFICAR',  # Renombrar
}

def aplicar_correcciones_manuales(nombre):
    if nombre in correcciones_manuales:
        return correcciones_manuales[nombre]
    return nombre

df_bucaramanga_clean['NOMBRE COMUNA'] = df_bucaramanga_clean['NOMBRE COMUNA'].apply(aplicar_correcciones_manuales)

if 'df_agregado' in globals():
    df_agregado['NOMBRE COMUNA'] = df_agregado['NOMBRE COMUNA'].apply(aplicar_correcciones_manuales)

if 'df_completo' in globals():
    df_completo['NOMBRE COMUNA'] = df_completo['NOMBRE COMUNA'].apply(aplicar_correcciones_manuales)

if 'df_powerbi' in globals():
    df_powerbi['COMUNA'] = df_powerbi['COMUNA'].apply(aplicar_correcciones_manuales)

print("✓ Correcciones manuales aplicadas")

# Verificar resultado final
print("\n" + "="*60)
print("NOMBRES FINALES CORREGIDOS")
print("="*60)

comunas_finales = sorted(df_bucaramanga_clean['NOMBRE COMUNA'].unique())

# Separar por tipo
comunas_urbanas = []
corregimientos = []
sin_geo = []

for comuna in comunas_finales:
    conteo = (df_bucaramanga_clean['NOMBRE COMUNA'] == comuna).sum()
    if 'Corregimiento' in comuna:
        corregimientos.append((comuna, conteo))
    elif comuna == 'SIN GEOCODIFICAR':
        sin_geo.append((comuna, conteo))
    else:
        comunas_urbanas.append((comuna, conteo))

print("\n✅ COMUNAS URBANAS (17):")
for i, (comuna, conteo) in enumerate(comunas_urbanas, 1):
    print(f"  {i:2}. '{comuna}' ({conteo:,} delitos)")

print(f"\n📍 CORREGIMIENTOS (3):")
for i, (corr, conteo) in enumerate(corregimientos, 1):
    print(f"  {i}. '{corr}' ({conteo:,} delitos)")

if sin_geo:
    print(f"\n⚠️  SIN GEOCODIFICAR:")
    for otro, conteo in sin_geo:
        print(f"  - '{otro}' ({conteo:,} delitos)")

# Validación final con ArcGIS
comunas_arcgis_oficiales = [
    'Comuna Norte', 'Comuna Nororiental', 'Comuna San Francisco',
    'Comuna Occidental', 'Comuna Garcia Rovira', 'Comuna La Concordia',
    'Comuna Ciudadela Real de Minas', 'Comuna Suroccidente', 'Comuna La Pedregosa',
    'Comuna Provenza', 'Comuna Sur', 'Comuna Cabecera del Llano',
    'Comuna Oriental', 'Comuna Morrorico', 'Comuna Centro',
    'Comuna Tejar', 'Comuna Mutis'
]

print("\n" + "="*60)
print("VALIDACIÓN FINAL CON ARCGIS")
print("="*60)

comunas_solo_nombres = [c[0] for c in comunas_urbanas]
comunas_correctas = [c for c in comunas_solo_nombres if c in comunas_arcgis_oficiales]
comunas_incorrectas = [c for c in comunas_solo_nombres if c not in comunas_arcgis_oficiales]

print(f"✅ Comunas que coinciden: {len(comunas_correctas)}/17")
for c in comunas_correctas:
    print(f"   ✓ {c}")

if comunas_incorrectas:
    print(f"\n⚠️  Comunas que NO coinciden: {len(comunas_incorrectas)}")
    for c in comunas_incorrectas:
        print(f"   ✗ {c}")
else:
    print("\n🎉 ¡PERFECTO! Todas las comunas coinciden con ArcGIS")

# Crear columna TIPO_ZONA
print("\n" + "="*60)
print("CREANDO COLUMNA: TIPO_ZONA")
print("="*60)

def clasificar_zona(nombre_comuna):
    if 'Corregimiento' in nombre_comuna:
        return 'RURAL'
    elif nombre_comuna == 'SIN GEOCODIFICAR':
        return 'SIN UBICACIÓN'
    else:
        return 'URBANA'

df_bucaramanga_clean['TIPO_ZONA'] = df_bucaramanga_clean['NOMBRE COMUNA'].apply(clasificar_zona)

print("\n✅ Distribución por TIPO_ZONA:")
for zona in ['URBANA', 'RURAL', 'SIN UBICACIÓN']:
    total = (df_bucaramanga_clean['TIPO_ZONA'] == zona).sum()
    pct = total / len(df_bucaramanga_clean) * 100
    print(f"   {zona}: {total:,} ({pct:.2f}%)")



In [ ]:
import pandas as pd
import numpy as np
from itertools import product

print("="*60)
print("FASE 4: GENERAR TABLA COMPLETA PARA POWER BI")
print("="*60)

# 1. CREAR TODAS LAS COMBINACIONES POSIBLES
print("\n[1/5] Creando todas las combinaciones posibles...")

# Extraer valores únicos de cada dimensión
comunas = sorted(df_bucaramanga_clean['NOMBRE COMUNA'].unique())
categorias = sorted(df_bucaramanga_clean['CATEGORIA DELITO'].unique())
dias = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
bloques = ['MADRUGADA', 'MAÑANA', 'TARDE', 'NOCHE']

print(f"  ✓ {len(comunas)} comunas")
print(f"  ✓ {len(categorias)} categorías")
print(f"  ✓ {len(dias)} días")
print(f"  ✓ {len(bloques)} bloques horarios")

# Crear todas las combinaciones
combinaciones = list(product(comunas, categorias, dias, bloques))
print(f"\n  ✓ Total combinaciones: {len(combinaciones):,}")

# Crear DataFrame con todas las combinaciones
df_completo = pd.DataFrame(combinaciones, columns=[
    'NOMBRE COMUNA',
    'CATEGORIA DELITO',
    'DIA NOMBRE',
    'BLOQUE_HORARIO'
])

# 2. AGREGAR DIA_SEMANA_NUM
print("\n[2/5] Agregando features adicionales...")

dia_a_num = {
    'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3,
    'viernes': 4, 'sábado': 5, 'domingo': 6
}
df_completo['DIA_SEMANA_NUM'] = df_completo['DIA NOMBRE'].map(dia_a_num)
print("  ✓ DIA_SEMANA_NUM agregado")

# 3. HACER MERGE CON DATOS HISTÓRICOS
print("\n[3/5] Combinando con datos históricos...")

df_completo = df_completo.merge(
    df_agregado[[
        'NOMBRE COMUNA', 'CATEGORIA DELITO', 'DIA NOMBRE', 'BLOQUE_HORARIO',
        'TOTAL_DELITOS', 'PROMEDIO_COMUNA', 'PROMEDIO_BLOQUE',
        'PROMEDIO_CATEGORIA', 'PROMEDIO_DIA'
    ]],
    on=['NOMBRE COMUNA', 'CATEGORIA DELITO', 'DIA NOMBRE', 'BLOQUE_HORARIO'],
    how='left'
)

# Rellenar valores faltantes con 0 para TOTAL_DELITOS
df_completo['TOTAL_DELITOS'].fillna(0, inplace=True)

# Para combinaciones sin historial, usar promedios globales
for col in ['PROMEDIO_COMUNA', 'PROMEDIO_BLOQUE', 'PROMEDIO_CATEGORIA', 'PROMEDIO_DIA']:
    # Calcular promedio global si falta
    promedio_global = df_agregado[col].mean()
    df_completo[col].fillna(promedio_global, inplace=True)

print(f"  ✓ Combinaciones con historial: {(df_completo['TOTAL_DELITOS'] > 0).sum():,}")
print(f"  ✓ Combinaciones sin historial: {(df_completo['TOTAL_DELITOS'] == 0).sum():,}")

# 4. CODIFICAR VARIABLES PARA PREDICCIÓN
print("\n[4/5] Preparando datos para predicción...")

# FIX: Re-fit the LabelEncoder for 'NOMBRE COMUNA' to include all cleaned comuna names.
# This is necessary because the cleaning of comuna names in cell n2Q7Tej6pCeA happened
# after the initial fitting of label_encoders in cell TsZIbrZvZu5m.
if 'NOMBRE COMUNA' in label_encoders:
    label_encoders['NOMBRE COMUNA'].fit(comunas) # 'comunas' list already contains the unique, cleaned names from df_bucaramanga_clean

# Codificar usando los mismos encoders del entrenamiento
for col in ['NOMBRE COMUNA', 'CATEGORIA DELITO', 'DIA NOMBRE', 'BLOQUE_HORARIO']:
    df_completo[f'{col}_ENCODED'] = label_encoders[col].transform(df_completo[col])

# Crear matriz de features para predicción
X_completo = df_completo[feature_columns]

print(f"  ✓ {len(X_completo):,} registros listos para predicción")

# 5. PREDECIR CON EL MODELO
print("\n[5/5] Realizando predicciones con el modelo...")

# Predecir categoría de riesgo
df_completo['RIESGO_PREDICHO_NUM'] = modelo_rf.predict(X_completo)
df_completo['CATEGORIA_RIESGO'] = le_target.inverse_transform(df_completo['RIESGO_PREDICHO_NUM'])

# Predecir probabilidades para cada clase
probabilidades = modelo_rf.predict_proba(X_completo)
df_completo['PROB_ALTO'] = probabilidades[:, 0]  # Probabilidad de ALTO
df_completo['PROB_BAJO'] = probabilidades[:, 1]  # Probabilidad de BAJO
df_completo['PROB_MEDIO'] = probabilidades[:, 2]  # Probabilidad de MEDIO

# Calcular probabilidad de la clase predicha
df_completo['PROBABILIDAD_PREDICCION'] = df_completo.apply(
    lambda row: row[f"PROB_{row['CATEGORIA_RIESGO']}"], axis=1
)

print("  ✓ Predicciones completadas")

# DISTRIBUCIÓN DE PREDICCIONES
print("\n" + "="*60)
print("DISTRIBUCIÓN DE RIESGO PREDICHO")
print("="*60)
print(df_completo['CATEGORIA_RIESGO'].value_counts())
print(f"\nPorcentajes:")
for cat in ['ALTO', 'MEDIO', 'BAJO']:
    pct = (df_completo['CATEGORIA_RIESGO'] == cat).sum() / len(df_completo) * 100
    print(f"  {cat}: {pct:.1f}%")

# CREAR TABLA FINAL PARA POWER BI
print("\n" + "="*60)
print("TABLA FINAL PARA POWER BI")
print("="*60)

# Seleccionar columnas relevantes para Power BI
df_powerbi = df_completo[[
    'NOMBRE COMUNA',
    'CATEGORIA DELITO',
    'DIA NOMBRE',
    'BLOQUE_HORARIO',
    'CATEGORIA_RIESGO',
    'PROBABILIDAD_PREDICHO',
    'PROB_ALTO',
    'PROB_MEDIO',
    'PROB_BAJO',
    'TOTAL_DELITOS'
]].copy()

# Renombrar para claridad en Power BI
df_powerbi.columns = [
    'COMUNA',
    'TIPO_DELITO',
    'DIA_SEMANA',
    'BLOQUE_HORARIO',
    'NIVEL_RIESGO',
    'CONFIANZA_PREDICCION',
    'PROBABILIDAD_ALTO',
    'PROBABILIDAD_MEDIO',
    'PROBABILIDAD_BAJO',
    'DELITOS_HISTORICOS'
]

print(f"Dimensiones: {df_powerbi.shape[0]:,} filas × {df_powerbi.shape[1]} columnas")
print(f"\nColumnas:")
for col in df_powerbi.columns:
    print(f"  - {col}")

# ESTADÍSTICAS FINALES
print("\n" + "="*60)
print("ESTADÍSTICAS DE LA TABLA")
print("="*60)
print(f"Total combinaciones: {len(df_powerbi):,}")
print(f"Combinaciones con historial: {(df_powerbi['DELITOS_HISTORICOS'] > 0).sum():,}")
print(f"Combinaciones nuevas (sin historial): {(df_powerbi['DELITOS_HISTORICOS'] == 0).sum():,}")
print(f"\nConfianza promedio de predicción: {df_powerbi['CONFIANZA_PREDICCION'].mean():.2%}")
print(f"Confianza mínima: {df_powerbi['CONFIANZA_PREDICCION'].min():.2%}")
print(f"Confianza máxima: {df_powerbi['CONFIANZA_PREDICCION'].max():.2%}")

# MUESTRA DE DATOS
print("\n" + "="*60)
print("MUESTRA - TOP 10 ZONAS DE MAYOR RIESGO")
print("="*60)
top_riesgo = df_powerbi[df_powerbi['NIVEL_RIESGO'] == 'ALTO'].nlargest(10, 'PROBABILIDAD_ALTO')
print(top_riesgo[['COMUNA', 'TIPO_DELITO', 'DIA_SEMANA', 'BLOQUE_HORARIO',
                   'PROBABILIDAD_ALTO', 'DELITOS_HISTORICOS']].to_string(index=False))

print("\n" + "="*60)
print("MUESTRA - TOP 10 ZONAS MÁS SEGURAS")
print("="*60)
top_seguro = df_powerbi[df_powerbi['NIVEL_RIESGO'] == 'BAJO'].nlargest(10, 'PROBABILIDAD_BAJO')
print(top_seguro[['COMUNA', 'TIPO_DELITO', 'DIA_SEMANA', 'BLOQUE_HORARIO',
                   'PROBABILIDAD_BAJO', 'DELITOS_HISTORICOS']].to_string(index=False))




In [ ]:
import pandas as pd
from datetime import datetime

print("="*60)
print("FASE 5: EXPORTAR ARCHIVOS PARA POWER BI")
print("="*60)

# Crear timestamp para los archivos
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 1. EXPORTAR TABLA DE PREDICCIONES
print("\n[1/2] Exportando tabla de predicciones...")

nombre_predicciones = f'predicciones_riesgo_bucaramanga_{timestamp}.csv'
df_powerbi.to_csv(nombre_predicciones, index=False, encoding='utf-8-sig')

print(f"  ✓ Archivo creado: {nombre_predicciones}")
print(f"  ✓ Tamaño: {len(df_powerbi):,} filas × {len(df_powerbi.columns)} columnas")
print(f"  ✓ Peso aproximado: {len(df_powerbi) * len(df_powerbi.columns) * 50 / 1024:.0f} KB")

# Resumen de la tabla de predicciones
print("\n  Resumen de PREDICCIONES:")
print(f"    - Total combinaciones: {len(df_powerbi):,}")
print(f"    - Nivel ALTO: {(df_powerbi['NIVEL_RIESGO'] == 'ALTO').sum():,} ({(df_powerbi['NIVEL_RIESGO'] == 'ALTO').sum()/len(df_powerbi)*100:.1f}%)")
print(f"    - Nivel MEDIO: {(df_powerbi['NIVEL_RIESGO'] == 'MEDIO').sum():,} ({(df_powerbi['NIVEL_RIESGO'] == 'MEDIO').sum()/len(df_powerbi)*100:.1f}%)")
print(f"    - Nivel BAJO: {(df_powerbi['NIVEL_RIESGO'] == 'BAJO').sum():,} ({(df_powerbi['NIVEL_RIESGO'] == 'BAJO').sum()/len(df_powerbi)*100:.1f}%)")

# 2. PREPARAR Y EXPORTAR DATOS HISTÓRICOS
print("\n[2/2] Exportando datos históricos completos...")

# Crear copia de datos históricos con columnas relevantes para Power BI
df_historico_powerbi = df_bucaramanga_clean[[
    'FECHA HECHO',
    'HORA',
    'BLOQUE_HORARIO',
    'DIA NOMBRE',
    'NOMBRE COMUNA',
    'CATEGORIA DELITO',
    'TIPO DELITO',
    'DELITO DETALLADO',
    'BARRIO',
    'LOCALIDAD',
    'GENERO',
    'EDAD',
    'CLASE SITIO',
    'ARMAS MEDIOS',
    'AÑO_NUM',
    'MES_NUM',
    'DIA_SEMANA_NUM',
    'ES_FIN_SEMANA'
]].copy()

# Renombrar columnas para Power BI (sin espacios ni caracteres especiales)
df_historico_powerbi.columns = [
    'FECHA',
    'HORA',
    'BLOQUE_HORARIO',
    'DIA_SEMANA',
    'COMUNA',
    'CATEGORIA_DELITO',
    'TIPO_DELITO',
    'DELITO_DETALLADO',
    'BARRIO',
    'LOCALIDAD',
    'GENERO',
    'EDAD',
    'CLASE_SITIO',
    'ARMAS_MEDIOS',
    'AÑO',
    'MES',
    'DIA_SEMANA_NUM',
    'ES_FIN_SEMANA'
]

nombre_historico = f'datos_historicos_bucaramanga_{timestamp}.csv'
df_historico_powerbi.to_csv(nombre_historico, index=False, encoding='utf-8-sig')

print(f"  ✓ Archivo creado: {nombre_historico}")
print(f"  ✓ Tamaño: {len(df_historico_powerbi):,} filas × {len(df_historico_powerbi.columns)} columnas")
print(f"  ✓ Peso aproximado: {len(df_historico_powerbi) * len(df_historico_powerbi.columns) * 50 / 1024 / 1024:.1f} MB")

# Resumen de datos históricos
print("\n  Resumen de DATOS HISTÓRICOS:")
print(f"    - Período: {df_historico_powerbi['FECHA'].min()} a {df_historico_powerbi['FECHA'].max()}")
print(f"    - Total delitos: {len(df_historico_powerbi):,}")
print(f"    - Comunas: {df_historico_powerbi['COMUNA'].nunique()}")
print(f"    - Categorías: {df_historico_powerbi['CATEGORIA_DELITO'].nunique()}")
print(f"    - Tipos de delito: {df_historico_powerbi['TIPO_DELITO'].nunique()}")

# RESUMEN FINAL
print("\n" + "="*60)
print("ARCHIVOS EXPORTADOS EXITOSAMENTE")
print("="*60)
print(f"\n📄 Archivo 1: {nombre_predicciones}")
print(f"   Uso: Mapa de riesgo con filtros en Power BI")
print(f"   Contiene: Predicciones ML para todas las combinaciones")
print(f"   Tamaño: ~500 KB")

print(f"\n📄 Archivo 2: {nombre_historico}")
print(f"   Uso: Análisis temporal, gráficos de tendencias")
print(f"   Contiene: Todos los delitos históricos (2016-2025)")
print(f"   Tamaño: ~{len(df_historico_powerbi) * 18 * 50 / 1024 / 1024:.0f} MB")

